# 0️⃣ **Module Import**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (46.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120880 files and direct

In [ ]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import platform
import nltk
import re
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from PIL import Image

In [ ]:
# 주관식 문항 답변 잘리지 않게
pd.set_option('display.max_colwidth', None)

#  1️⃣ **전처리**


## 1) 파일 불러오기

**GitHub 이용**

In [ ]:
# 데이터 파일 불러오기
# import pandas as pd
# LIS = pd.read_csv('')

**GoogleDrive 이용**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 파일 불러오기
LIS_2023origin = pd.read_csv('/content/drive/MyDrive/2023_LIS_SURVEY_응답.csv')
LIS_2023add = pd.read_csv('/content/drive/MyDrive/희망근무분야.csv')

LIS_2022 = pd.read_csv('/content/drive/MyDrive/LIS_SURVEY_2022.csv')
LIS_2021 = pd.read_csv('/content/drive/MyDrive/LIS_SURVEY_2021.csv')

## 2) 2021 데이터

In [ ]:
# 불필요한 column 정리
LIS_2021 = LIS_2021.drop('타임스탬프', axis=1)
LIS_2021 = LIS_2021.drop('이메일을 적어주세요. ', axis=1)
# 응답 오류 정리
LIS_2021.drop(51, inplace=True)
LIS_2021.drop(60, inplace=True)
LIS_2021.drop(88, inplace=True)
LIS_2021.reset_index(inplace=True)
LIS_2021.drop('index', axis=1, inplace=True)
LIS_2021.reset_index(inplace=True)

In [ ]:
LIS_2021.rename(columns = {'성별은 어떻게 되시나요?':'성별',
                      '졸업 여부는?':'졸업여부',
                      '입학년도가 어떻게 되시나요? (학번 앞 네자리)': '입학연도',
                      '현재 대학원생 인가요? ':'대학원생 여부',
                      'Q1) 입학 당시 희망 진로가 있었나요?':'희망진로유무_입학', #재학생
                      'Q1-1) 입학 당시 희망 진로는? (복수응답 가능)':'희망진로_입학',
                      'Q2) 현재 희망 진로가 있나요?':'희망진로유뮤_현재',
                      'Q2-1) 현재 희망 진로는? (복수응답 가능)':'희망진로_현재',
                      'Q2-2) 현재 희망 진로가 없다면 그 이유는 무엇인가요? (복수 응답 가능)':'현재희망진로없는이유',
                      'Q3) 입학당시와 현재의 희망진로가 다르다면, 그 이유가 무엇인가요?  (다르지 않을 경우 ‘다르지 않음’ 이라고 작성 후 그 이유도 서술해주세요)':'희망진로불일치이유',
                      'Q4) 평소 진로에 있어서 고민이 있다면 무엇인가요? (복수 응답 가능)':'평소진로고민',
                      'Q5) 평소 진로 관련 고민을 어떤 방식으로 해결하나요? (복수 응답 가능)':'진로고민해결방법',
                      'Q6) 진로를 선택할 때 중요하게 생각하는 가치는?  [1순위]':'진로가치_1순위',
                      'Q6) 진로를 선택할 때 중요하게 생각하는 가치는?  [2순위]':'진로가치_2순위',
                      'Q6) 진로를 선택할 때 중요하게 생각하는 가치는?  [3순위]':'진로가치_3순위',
                      'Q7) 희망 진로를 위해 현재 준비하고 있는 일이 있나요? (ex 어학, 학점, 대내외 교외활동, 스터디 등)':'현재준비하고있는일_유무',
                      'Q7-1) 희망 진로를 위해 현재 준비하고 있는 일이 무엇인가요? (복수 응답 가능)':'현재준비하고있는일_내용',
                      'Q7-2) 현재 준비하고 있는 일이 없다면 그 이유는 무엇인가요? (복수 응답 가능)':'현재준비하고있는일없는이유',
                      'Q8) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하나요? (자유롭게 서술해 주세요)':'문헌정보학가능진로종류',
                      'Q9) 다전공 이수의 장점은? (복수 선택 가능)':'다전공이수장점',
                      'Q10) 다전공 이수의 단점은? (복수 선택 가능)':'다전공이수단점',
                      'Q11) 현재 이수 중인 다전공(복수, 융합, 연계, 부전공, 자기설계)이 있습니까? ':'다전공이수_여부',
                      'Q11-1-1) 이수 중인 다전공은? (문헌정보 복전생일 경우, 주전공을 적어주시면 됩니다)':'다전공이수_종류',
                      'Q11-1-2) 해당 다전공을 선택한 이유는 무엇인가요?':'다전공선택이유',
                      'Q11-1-3) 다전공을 이수하면서 겪은 어려움 혹은 문제가 있다면 무엇인가요?':'다전공이수어려움',
                      'Q12) 문헌정보학과 교육과정에서 가장 흥미있는 주제는? ':'문헌정보학흥미주제',
                      'Q13) 본 학과에 추가로 개설 되었으면 하는 교육과정이 있나요?':'추가개설희망과정',
                      'Q14)  대학원 진학을 고려하고 있나요? ':'대학원진학희망_여부',
                      'Q14-1-1) 어떤 대학원에 진학할 생각이신가요? (복수 응답 가능)':'대학원진학희망_종류',
                      'Q14-1-2) 예상 대학원 진학 시기는?':'예상대학원시기',
                      'Q14-1-3) 대학원 진학의 목적은 무엇인가요? (복수 응답 가능)':'대학원진학_목적',
                      'Q14-2) 대학원 입학을 고려하지 않는 이유는? (복수 응답 가능)':'대학원진학미고려_이유',
                      'Q15) 향후 가장 근무하고 싶은 분야가 어디인가요?':'희망근무분야',
                      'Q15-1) 희망 직종은 무엇인가요? (복수 응답 가능)':'희망직무내용',
                      'Q15-2) 근무하고 싶은 도서관 유형은? (복수 선택 가능) ':'희망근무도서관유형',
                      'Q15-3) 희망 직무 계열은 무엇인가요? ':'희망직무계열',
                      'Q16) 희망 연봉 수준은 어느 정도인가요?':'희망연봉수준',
                      'Q17) 예상 취업 준비 기간은 어느 정도로 생각하시나요? ':'예상취업준비기간',
                      'Q18) 본 학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하나요?':'필요취업강화프로그램',
                      'Q19) 평소 취업 관련 정보를 얻는 방법은 무엇인가요? (복수 응답 가능) ':'취업정보수집방법',
                      'Q20) 현재 문헌정보학과에서 취업을 위한 정보 공유가 적절히 이루어지고 있다고 생각하시나요?':'학과취업정보교류정도',
                      'Q21) 재학생들에게 취업정보를 제공한다면, 어떤 방법을 선호하나요?  (복수 응답 가능)':'선호취업정보공유방법',
                      'Q22) 재학생들과 졸업생들이 정보 교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수응답 가능)':'학과정보교류선호방법',

                      '졸업연도는? (연도 숫자만 적어주세요)':'졸업연도',#취업자
                      '현재 취직을 하셨나요? ':'취직여부',
                      'Q1) 현재 근무 중인 분야가 어디인가요? ':'현재근무분야',
                      'Q1-1) 근무지는 어디인가요?':'근무지',
                      'Q1-2) 직무 내용은 무엇인가요?':'직무내용',
                      'Q1-3) 직무 계열이 무엇인가요?':'직무계열',
                      'Q2) 직무 만족도는 어떠한가요?':'직무만족도',
                      'Q3) 현재 직무의 연봉 수준은 어느정도 인가요?':'연봉수준',
                      'Q4) 취직 전(학부시절 포함) 진로 선택에 있어 중요시했던 가치는 무엇인가요? [1순위]':'취직전진로가치_1순위',
                      'Q4) 취직 전(학부시절 포함) 진로 선택에 있어 중요시했던 가치는 무엇인가요? [2순위]':'취직전진로가치_2순위',
                      'Q4) 취직 전(학부시절 포함) 진로 선택에 있어 중요시했던 가치는 무엇인가요? [3순위]':'취직전진로가치_3순위',
                      'Q5) 취직 전후로 직업 가치관의 변화가 있었나요?':'취직전후진로가치관변화_여부',
                      '취직 후  중요시 하게 된 가치는 무엇인가요? [1순위]':'취직후진로가치_1순위',
                      '취직 후  중요시 하게 된 가치는 무엇인가요? [2순위]':'취직후진로가치_2순위',
                      '취직 후  중요시 하게 된 가치는 무엇인가요? [3순위]':'취직후진로가치_3순위',
                      'Q6) 취업 준비기간은 어떻게 되시나요? ': '취업준비기간',
                      'Q7) 과거 취업을 위해 어떤 준비를 하셨나요? (복수 응답 가능)':'과거취업준비내용',
                      'Q8) 과거 취업 준비 시 힘든 것이 무엇이었습니까? (복수 응답 가능)':'과거취업준비시어려움',
                      'Q9) 현재의 근무지를 본 학과 후배들에게 추천할 의향이 있나요?':'현재근무지후배추천의향',
                      'Q10) 대학원에 진학하셨나요? (전공 및 학교 무관)':'대학원진학_여부2',
                      'Q10-1-1) 어떤 대학원에 진학하셨나요?':'대학원진학_종류2',
                      'Q10-1-2) 대학원 진학 시기는 언제인가요?':'대학원진학시기2',
                      'Q10-1-3) 대학원 진학의 목적은 무엇이었나요? (복수 응답 가능)':'대학원진학_목적2',
                      'Q10-2-1) 대학원 진학을 고려하지 않은 이유는 무엇이었나요? (복수 응답 가능)':'대학원진학미고려_이유2',
                      'Q10-2-2) 향후 대학원을 진학할 의향이 있으신가요?':'향후대학원진학의향_유무2',
                      'Q10-2-3) 진학할 의향이 있다면 어떤 대학원에 진학할 생각이신가요? (복수 응답 가능)':'향후대학원진학의향_종류',
                      'Q11) 취직에 다전공 이수가 유리하다고 생각하시나요?':'다전공이수유리정도',
                      'Q12) 다전공 이수의 장점은? (복수 선택 가능)':'다전공이수장점2',
                      'Q13) 다전공 이수의 단점은? (복수 선택 가능)':'다전공이수단점2',
                      'Q14) 재학 당시 다전공을 이수했나요?':'다전공이수_여부2',
                      'Q14-1-1)재학당시 이수 한 다전공은 무엇인가요? (문헌정보 복전생일 경우, 주전공을 적어주시면 됩니다)':'다전공이수_종류2',
                      'Q14-1-2)이수한 다전공을 선택한 이유는 무엇입니까?':'다전공선택이유2',
                      'Q14-1-3) 다전공을 이수하면서 겪은 어려움 혹은 문제가 있었다면 무엇인가요?':'다전공이수어려움2',
                      'Q14-2) 재학 당시 다전공을 이수하지 않은 이유가 무엇입니까?':'다전공미이수이유2',
                      'Q15) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하시나요?':'문헌정보학가능진로종류2',
                      'Q16) 본 학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하시나요?':'필요취업강화프로그램2',
                      'Q17) 문헌정보학과 교육과정에서 가장 흥미있는 주제는 무엇이었나요? ':'문헌정보학흥미주제2',
                      'Q18) 문헌정보학과 재학 당시 배웠으면 좋았을 것 같은 교과 과정이 있다면 무엇인가요? ':'개설희망과정2',
                      'Q19) 문헌정보학과에서 취업을 위한 정보공유가 적절히 이루어졌다고 생각하시나요?':'학과취업정보교류정도2',
                      'Q20) 재학생들과 졸업생들이 정보 교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수응답 가능)':'학과정보교류선호방법2',

                      'Q1) 희망 근무 분야가 어디인가요?':'희망근무분야3', #취준생
                      'Q1-1) 희망 근무지는 어디인가요?':'희망근무지3',
                      'Q1-2) 희망 직무 내용은 무엇인가요? (복수 응답 가능)':'희망직무내용3',
                      'Q1-3) 희망 직무 계열이 무엇인가요?':'희망직무계열3',
                      'Q2) 희망 연봉 수준은 어느정도 인가요?':'희망연봉수준3',
                      'Q3) 예상 취업 준비 기간은 어느 정도로 생각하시나요? ':'예상취업준비기간3',
                      'Q4) 현재 취업을 위해 어떤 준비를 하고 계시나요? (복수 응답 가능)':'현재취업준비내용',
                      'Q5) 현재 취업 준비 중에 가장 힘든 것은 무엇인가요? (복수 응답 가능)':'현재취업준비시어려움',
                      'Q6) 진로를 선택할 때 중요하게 생각하는 가치가 무엇인가요? [1순위]':'진로가치_1순위3',
                      'Q6) 진로를 선택할 때 중요하게 생각하는 가치가 무엇인가요? [2순위]':'진로가치_2순위3',
                      'Q6) 진로를 선택할 때 중요하게 생각하는 가치가 무엇인가요? [3순위]':'진로가치_3순위3',
                      'Q7) 대학원에 진학하셨나요?':'대학원진학_여부3',
                      'Q7-1-1) 어떤 대학원에 진학하셨나요?':'대학원진학_종류3',
                      'Q7-1-2) 대학원 진학 시기는 언제인가요?':'대학원진학시기3',
                      'Q7-1-3) 대학원 진학의 목적은 무엇이었나요? (복수 응답 가능)':'대학원진학_목적3',
                      'Q7-2-1)대학원 진학을 고려하지 않은 이유는 무엇인가요? (복수 응답 가능)':'대학원진학미고려_이유3',
                      'Q7-2-2) 향후 대학원에 진학할 의향이 있으신가요?':'향후대학원진학의향_유무3',
                      'Q7-2-3) 진학할 의향이 있다면 어떤 대학원에 진학할 생각이신가요? (복수 응답 가능)':'향후대학원진학의향_종류',
                      'Q8) 취직에 다전공 이수가 유리하다고 생각하시나요?':'다전공이수유리정도',
                      'Q9) 다전공 이수의 장점은? (복수 선택 가능).1':'다전공이수장점3',
                      'Q10) 다전공 이수의 단점은? (복수 선택 가능).1':'다전공이수단점3',
                      'Q11) 재학 당시 다전공을 이수하셨나요?':'다전공이수_여부3',
                      'Q11-1-1) 재학 당시 이수한 다전공은 무엇인가요? (문헌정보 복전생일 경우, 주전공을 적어주시면 됩니다)':'다전공이수_종류3',
                      'Q11-1-2) 이수한 다전공을 선택한 이유는 무엇인가요?':'다전공선택이유3',
                      'Q11-1-3) 다전공을 이수하면서 겪은 어려움 혹은 문제가 있었다면 무엇인가요?':'다전공이수어려움3',
                      'Q11-2) 재학 당시 다전공을 이수하지 않은 이유는 무엇인가요?':'다전공미이수이유3',
                      'Q12) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하시나요?':'문헌정보가능진로종류3',
                      'Q13) 본 학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하시나요?':'필요취업강화프로그램3',
                      'Q14) 문헌정보학과 교육과정에서 가장 흥미있는 주제는 무었이었나요? ':'문헌정보학흥미주제3',
                      'Q15) 문헌정보학과 재학 당시 배웠으면 좋았을 것 같은 교과 과정이 있다면 무엇인가요?':'개설희망과정3',
                      'Q16) 문헌정보학과에서 취업을 위한 정보공유가 적절히 이루어졌다고 생각하시나요?':'학과취업정보교류정도3',
                      'Q17) 재학생들과 졸업생들이 정보교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수응답 가능)':'학과정보교류선호방법3'},inplace=True)

In [ ]:
## 학부 재학생 데이터 프레임
Ustudent_2021=LIS_2021[(LIS_2021['졸업여부']=='재학생 (대학원생 포함)')&(LIS_2021['대학원생 여부']=='아니오')]
Ustudent_2021=Ustudent_2021.loc[:,:'학과정보교류선호방법']
Ustudent_2021.reset_index(drop=True,inplace=True)

## 대학원 재학생 데이터 프레임
Gstudent_2021=LIS_2021[(LIS_2021['졸업여부']=='재학생 (대학원생 포함)')&(LIS_2021['대학원생 여부']=='예 (2021년 입학예정 포함)')]
Gstudent_2021=Gstudent_2021.loc[:,:'학과정보교류선호방법']
Gstudent_2021.reset_index(drop=True,inplace=True)

## 재학생(학부+대학원) 데이터 프레임 --- 추가
student_2021 = LIS_2021[LIS_2021['졸업여부']=='재학생 (대학원생 포함)']

## 취업자 데이터 프레임
worker_2021=LIS_2021[(LIS_2021['졸업여부']=='졸업생')&(LIS_2021['취직여부']=='예')]
temp=worker_2021[['성별']]
worker_2021=worker_2021
worker_2021=pd.concat([temp,worker_2021],axis=1).reset_index(drop=True)

## 취준생 데이터 프레임
graduate=LIS_2021[LIS_2021['졸업여부']=='졸업생']
temp1=graduate.loc[:,'졸업연도']
temp2=graduate.loc[:,'희망근무분야3':'학과정보교류선호방법3']
temp1=pd.DataFrame(temp1)
seeker_2021=pd.concat([temp,temp1,temp2],axis=1)
seeker_2021.dropna(subset = ['희망근무분야3'],inplace=True) #희망근무분야 nan = 취업자임! 그래서 이거 기준으로 드롭함
seeker_2021.reset_index(drop=True,inplace=True)

## 3) 2022 데이터

In [ ]:
# 불필요한 column 정리
LIS_2022 = LIS_2022.drop('타임스탬프', axis=1)
LIS_2022 = LIS_2022.drop('이메일을 적어주세요. ', axis=1)

In [ ]:
LIS_2022.rename(columns = {
                      '성별은 어떻게 되시나요?':'성별',
                      '졸업 여부는 어떻게 되시나요?':'졸업여부',
                      '입학연도가 어떻게 되시나요? (학번 앞 네자리)': '입학연도',
                      '현재 대학원생 인가요? ': '대학원생 여부',
                      '문헌정보가 아닌 다른 주전공을 이수하고 있습니까? ':'주전공',

                      # 재학생 개별질문
                      'U011) 입학 당시 희망 진로가 있었나요?':'희망진로유무_입학',
                      'U011.1) 입학 당시 희망 진로는 무엇이었나요? (복수응답 가능)':'희망진로_입학',
                      'U012) 현재 희망 진로가 있나요?':'희망진로유뮤_현재',
                      'U012.1) 현재 희망 진로는 무엇인가요? (복수응답 가능)':'희망진로_현재',
                      'U012.2) 현재 희망 진로가 없다면 그 이유는 무엇인가요? (복수 응답 가능)':'현재희망진로없는이유',
                      'U013) 입학당시와 현재의 희망진로가 다르다면, 그 이유가 무엇인가요?  ':'희망진로불일치이유',
                      'U014)입학당시와 현재의 희망진로가 달라진 것에 영향을 준 것은 무엇인가요? ':'희망진로불일치_영향',
                      'U021) 평소 진로에 있어서 고민이 있다면 무엇인가요? (복수 응답 가능)':'평소진로고민',
                      'U022) 평소 진로 관련 고민을 어떤 방식으로 해결하나요? (복수 응답 가능)':'진로고민해결방법',
                      # 재학생 - 학과 관련 질문
                      'U111) 문헌정보학과 교육과정에서 가장 흥미있는 주제는 무엇인가요? ':'문헌정보학흥미주제',
                      'U112) 문헌정보학과에 추가로 개설 되었으면 하는 교육과정이 있다면 서술해주세요.':'추가개설희망과정',
                      'U121) 다전공 이수의 장점은? (복수 응답 가능)':'다전공이수장점',
                      'U122) 다전공 이수의 단점은 무엇인가요? (복수 응답 가능)':'다전공이수단점',
                      'U123) 현재 이수 중인 혹은 학부생 때 이수 했던 다전공(복수, 융합, 연계, 부전공, 자기설계)이 있나요?':'다전공이수_여부',
                      'U123.1) 이수 중인 혹은 이수했던 다전공은 무엇인가요?':'다전공이수_종류',
                      'U123.2) 자기설계를 선택한 경우, 전공을 구성하는 학과를 모두 적어주세요.':'자기설계구성학과', #2022 추가
                      'U123.3) 다전공 명칭이 변경, 통폐합 등으로 위의 다전공 목록에 해당 사항 없는 경우 이수한 다전공을 직접 적어주세요':'다전공명칭통폐합', #2022 추가
                      'U123.4) 해당 다전공을 선택한 이유는 무엇인가요?':'다전공선택이유',
                      'U123.5) 다전공을 이수하면서 겪은 어려움 혹은 문제가 있다면 무엇인가요? (복수 응답 가능)':'다전공이수어려움',
                      'U123.6)다전공을 이수하지 않은 이유가 무엇인가요?':'다전공미이수이유',
                      'U131) 현재 문헌정보학과에서 취업을 위한 정보 공유가 적절히 이루어지고 있다고 생각하시나요?':'학과취업정보교류정도',
                      'U132) 평소 취업 관련 정보를 얻는 방법은 무엇인가요? (복수 응답 가능) ':'취업정보수집방법',
                      'U133) 재학생들과 졸업생들이 정보 교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수응답 가능)':'학과정보교류선호방법',
                      'U134) 선호하는 취업정보 공유 방법은 무엇인가요? (복수 응답 가능)':'선호취업정보공유방법',
                      'U135) 문헌정보학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하나요? (복수 응답 가능)':'필요취업강화프로그램',
                      'U136) 문헌정보학과에 진로 관련 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하시나요? (복수 응답 가능)':'필요진로관련프로그램', #2022 추가
                      'U137) 진로관련 프로그램과 관련하여 앞의 선지에 언급되지 않은 희망 프로그램을 자유롭게 서술해주세요. (자세한 내용이어도 좋습니다.)':'희망진로관련프로그램', #2022 추가

                      #재학생 - 취업 관련 질문
                      'U211) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하나요? (복수 응답 가능)':'문헌정보학가능진로종류',
                      'U212) 문헌정보학 전공자가 가질 수 있는 진로에 대해서 앞의 선지에 언급되지 않은 내용을 자유롭게 서술해 주세요. (자세한 내용이어도 좋습니다.)':'문헌정보학희망진로종류', #2022 추가
                      'U213) 희망 직종은 무엇인가요? ' : '희망근무분야',
                      'U213.1) 근무하고 싶은 도서관 유형은 무엇인가요? (복수 응답 가능) ':'희망근무도서관유형',
                      'U213.2) 희망 직종은 무엇인가요? (복수 응답 가능)':'희망직무내용',
                      'U213.3) 희망 직무 계열은 무엇인가요? ':'희망직무계열',
                      'U221) 희망 연봉 수준은 어느 정도인가요?':'희망연봉수준',
                      'U231) 예상 취업 준비 기간은 어느 정도로 생각하시나요? ':'예상취업준비기간',
                      'U232) 희망 진로와 취업을 위해 현재 준비하고 있는 일이 있나요? (ex 어학, 학점, 대내외 교외활동, 스터디 등)':'현재준비하고있는일_유무',
                      'U232.1) 희망 진로를 위해 현재 준비하고 있는 일이 무엇인가요? (복수 응답 가능)':'현재준비하고있는일_내용',
                      'U232.2) 현재 준비하고 있는 일이 없다면 그 이유는 무엇인가요? (복수 응답 가능)':'현재준비하고있는일없는이유',
                      'U241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요? [1순위]':'진로가치_1순위',
                      'U241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요? [2순위]':'진로가치_2순위',
                      'U241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요? [3순위]':'진로가치_3순위',
                      # 재학생 - 대학원 관련 질문
                      'U311)  대학원 진학을 고려하고 있나요? ':'대학원진학희망_여부',
                      'U311.1) 어떤 대학원에 진학하고 싶나요?(진학하셨나요?)':'대학원진학희망_종류',
                      'U311.2) 희망(실제) 대학원 진학 시기는 언제인가요?':'예상대학원시기',
                      'U311.3) 대학원 진학의 목적은 무엇인가요?(무엇이었나요?) (복수 응답 가능)':'대학원진학_목적',
                      'U311.4) 대학원 진학 준비 중(준비 당시) 어려웠던 점은 무엇인가요? (복수 응답 가능)':'대학원진학어려움', #2022 추가
                      'U311.5) 대학원 진학을 고려하지 않는 이유는 무엇이었나요? (복수 응답 가능)':'대학원진학미고려_이유',



                      '졸업연도가 어떻게 되시나요? (학부 기준)':'졸업연도',
                      '현재 취직을 하셨나요? ':'취직여부',
                      'W213) 현재 근무하고 있는 분야가 무엇인가요?':'현재근무분야',
                      ' W213.1) 근무하고 있는 도서관 유형은 무엇인가요? (복수 응답 가능) ':'근무지2',
                      'W213.2) 현재 근무 중인 직종은 무엇인가요? (복수 응답 가능)':'직무내용2',
                      'W213.3) 현재 근무 중인 직무 계열은 무엇인가요?':'직무계열2',
                      'W251) 현재 직무 만족도는 어떠한가요?':'직무만족도2',
                      'W221) 실제 연봉 수준은 어느 정도인가요?':'연봉수준2',
                      'W241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [1순위]':'취직전진로가치_1순위',
                      'W241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [2순위]':'취직전진로가치_2순위',
                      'W241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [3순위]':'취직전진로가치_3순위',
                      'W242) 취직 전후로 직업 가치관의 변화가 있었나요?':'취직전후진로가치관변화_여부',
                      'W242.1) 취직 후 중요시 하게 된 가치는 무엇인가요? [1순위]':'취직후진로가치_1순위',
                      'W242.1) 취직 후 중요시 하게 된 가치는 무엇인가요? [2순위]':'취직후진로가치_2순위',
                      'W242.1) 취직 후 중요시 하게 된 가치는 무엇인가요? [3순위]':'취직후진로가치_3순위',
                      'W231) 실제 취업 준비 기간은 어느 정도이었나요?': '취업준비기간2',
                      'W232) 희망 진로와 취업을 위해 과거에 준비했던 일이 있나요?' : '과거취업준비내용여부', #2022 추가
                      'W232.1) 희망 진로와 취업을 위해  준비했던 일이 무엇인가요? (복수 응답 가능)':'과거취업준비내용',
                      'W232.2) 현재 준비하고 있는 일이 없다면(과거에 없었다면) 그 이유는 무엇인가요? (복수 응답 가능)' : '취업준비없는이유2', #2022 추가
                      'W233) 취업 준비 중에 가장 힘들었던 것은 무엇인가요? (복수 응답 가능)':'과거취업준비시어려움',
                      'W252) 현재의 근무지를 문헌정보학과 후배들에게 추천할 의향이 있나요?':'현재근무지후배추천의향',

                      'W311) 대학원에 진학했나요?':'대학원진학_여부2',
                      'W311.1) 어떤 대학원에 진학하셨나요?':'대학원진학_종류2',
                      ' W311.2) 대학원 진학 시기는 언제인가요?':'대학원진학시기2',
                      'W311.3) 대학원 진학의 목적은 무엇이었나요? (복수 응답 가능)':'대학원진학_목적2',
                      'W311.4) 대학원 진학 준비 중 어려운, 어려웠던 점은 무엇인가요? (복수 응답 가능)':'대학원진학_어려움2', #2022 추가
                      'W311.5) 대학원 진학을 고려하지 않은 이유는 무엇이었나요? (복수 응답 가능)':'대학원진학미고려_이유2',
                      'W311.6) 향후 대학원에 진학할 의향이 있으신가요?':'향후대학원진학의향_유무2',
                      'W311.7) 진학할 의향이 있다면 어떤 대학원에 진학할 생각이신가요? (복수 응답 가능)':'향후대학원진학의향_종류2',
                      'W123) 취직에 다전공 이수가 유리하다고 생각하시나요?':'다전공이수유리정도2',
                      'W121) 다전공 이수의 장점은 무엇인가요? (복수 응답 가능)':'다전공이수장점2',
                      'W122) 다전공 이수의 단점은 무엇인가요? (복수 응답 가능)':'다전공이수단점2',
                      'W124) 재학 당시 이수한 다전공(복수, 융합, 연계, 부전공)이 있나요?':'다전공이수_여부2',
                      'W124.1) 재학당시 이수한 다전공은 무엇인가요?':'다전공이수_종류2',
                      'W124.2) 자기설계를 선택한 경우, 전공을 구성하는 학과를 모두 적어주세요.':'자기설계구성학과2', #2022 추가 질문
                      'W124.3) 다전공 명칭이 변경, 통폐합 등으로 위의 다전공 목록에 해당 사항 없는 경우 이수한 다전공을 직접 적어주세요':'다전공명칭통폐합2', #2022 추가
                      'W124.4) 해당 다전공을 선택한 이유는 무엇인가요?':'다전공선택이유2',
                      'W124.5) 다전공을 이수하면서 겪은 어려움 혹은 문제가 있었다면 무엇인가요? (복수 응답 가능)':'다전공이수어려움2',
                      'W124.6) 재학 당시 다전공을 이수하지 않은 이유가 무엇인가요?':'다전공미이수이유2',
                      'W211) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하나요? ':'문헌정보학가능진로종류2',
                      'W212) 문헌정보학 전공자가 가질 수 있는 진로에 대해서 앞의 선지에 언급되지 않은 내용을 자유롭게 서술해 주세요. (자세한 내용이어도 좋습니다.)':'문헌정보학희망진로종류2', #2022 추가
                      'W135) 문헌정보학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하나요? (복수 응답 가능)':'필요취업강화프로그램2',
                      'W136) 문헌정보학과에 진로 관련 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하시나요? (복수 응답 가능)':'필요진로관련프로그램2', #2022 추가
                      'W111) 문헌정보학과 교육과정에서 가장 흥미있는 주제는 무엇이었나요?':'문헌정보학흥미주제2',
                      'W112) 재학 당시 문헌정보학과에 추가로 개설되었다면 좋았을 것 같은 교육과정에 대해서 서술해주세요.':'개설희망과정2',
                      'W132) 취업 관련 정보를 얻었던 방법은 무엇인가요? (복수 응답 가능)':'취업정보수집방법2',#2022 추가
                      'W131) 문헌정보학과에서 취업을 위한 정보 공유가 적절히 이루어졌다고 생각하시나요?':'학과취업정보교류정도2',
                      'W133) 재학생들과 졸업생들이 정보 교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수 응답 가능)':'학과정보교류선호방법2',
                      'W134) 선호하는 취업정보 공유 방법은 무엇인가요? (복수 응답 가능)':'선호취업정보공유방법2',#2022 추가
                      'W137) 진로관련 프로그램과 관련하여 앞의 선지에 언급되지 않은 희망 프로그램을 자유롭게 서술해주세요. (자세한 내용이어도 좋습니다.)':'희망진로관련프로그램2', #2022 추가



                      'S213) 희망 직종은 무엇인가요?':'희망근무분야3', #취준생
                      ' S213.1) 근무하고 싶은 도서관 유형은 무엇인가요? (복수 응답 가능) ':'희망근무지3',
                      'S213.2) 희망 직종은 무엇인가요? (복수 응답 가능)':'희망직무내용3',
                      'S213.3) 희망 직무 계열은 무엇인가요? ':'희망직무계열3',
                      'S221) 희망 연봉 수준은 어느 정도인가요?':'희망연봉수준3',
                      'S231) 예상 취업 준비 기간은 어느 정도로 생각하시나요?':'예상취업준비기간3',
                      'S232) 희망 진로와 취업을 위해 현재 준비하고 있는 일이 있나요?':'현재취업준비여부3', ##2022 추가 질문
                      'S232.1) 희망 진로와 취업을 위해 현재 준비하고 있는 일이 무엇인가요? (복수 응답 가능)':'현재취업준비내용3',
                      'S232.2) 현재 준비하고 있는 일이 없다면(과거에 없었다면) 그 이유는 무엇인가요? (복수 응답 가능)':'취업준비없는이유3', ##2022 추가 질문
                      'S233) 취업 준비 중에 가장 힘든 것은 무엇인가요? (복수 응답 가능)':'현재취업준비시어려움3',
                      'S241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [1순위]':'진로가치_1순위3',
                      'S241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [2순위]':'진로가치_2순위3',
                      'S241) 진로를 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [3순위]':'진로가치_3순위3',
                      'S311) 대학원에 진학했나요?':'대학원진학_여부3',
                      'S311.1) 어떤 대학원에 진학하셨나요?':'대학원진학_종류3',
                      ' S311.2) 대학원 진학 시기는 언제인가요?':'대학원진학시기3',
                      'S311.3) 대학원 진학의 목적은 무엇이었나요? (복수 응답 가능)':'대학원진학_목적3',
                      'S311.4) 대학원 진학 준비 중 어려운, 어려웠던 점은 무엇인가요? (복수 응답 가능)':'대학원진학_어려움3', ##2022 추가 질문
                      'S311.5) 대학원 진학을 고려하지 않은 이유는 무엇이었나요? (복수 응답 가능)':'대학원진학미고려_이유3',
                      'S311.6) 향후 대학원에 진학할 의향이 있으신가요?':'향후대학원진학의향_유무3',
                      'S311.7) 진학할 의향이 있다면 어떤 대학원에 진학할 생각이신가요? (복수 응답 가능)':'향후대학원진학의향_종류3',
                      'S123) 취직에 다전공 이수가 유리하다고 생각하시나요?':'다전공이수유리정도3',
                      's121) 다전공 이수의 장점은 무엇인가요? (복수 응답 가능)':'다전공이수장점3',
                      'S122) 다전공 이수의 단점은 무엇인가요? (복수 응답 가능)':'다전공이수단점3',
                      'S124) 재학 당시 이수한 다전공(복수, 융합, 연계, 부전공)이 있나요?':'다전공이수_여부3',
                      'S124.1) 재학당시 이수한 다전공은 무엇인가요?':'다전공이수_종류3',
                      'S124.2) 자기설계를 선택한 경우, 전공을 구성하는 학과를 모두 적어주세요.':'자기설계구성학과3', ##2022 추가 질문
                      'S124.3) 다전공 명칭이 변경, 통폐합 등으로 위의 다전공 목록에 해당 사항 없는 경우 이수한 다전공을 직접 적어주세요':'다전공명칭통폐합3', ##2022 추가 질문
                      'S124.4) 해당 다전공을 선택한 이유는 무엇인가요?':'다전공선택이유3',
                      'S124.5 다전공을 이수하면서 겪은 어려움 혹은 문제가 있다면 무엇인가요? (복수 응답 가능)':'다전공이수어려움3',
                      'S124.6) 재학 당시 다전공을 이수하지 않은 이유가 무엇인가요?':'다전공미이수이유3',
                      'S211) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하나요? ':'문헌정보가능진로종류3',
                      'S212) 문헌정보학 전공자가 가질 수 있는 진로에 대해서 앞의 선지에 언급되지 않은 내용을 자유롭게 서술해 주세요. (자세한 내용이어도 좋습니다.)':'문헌정보가능진로종류서술형3', ##2022 추가 질문
                      'S135) 문헌정보학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하나요? (복수 응답 가능)':'필요취업강화프로그램3',
                      'S136) 문헌정보학과에 진로 관련 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하시나요? (복수 응답 가능)':'필요진로관련프로그램3', ##2022 추가 질문
                      'S137) 진로관련 프로그램과 관련하여 앞의 선지에 언급되지 않은 희망 프로그램을 자유롭게 서술해주세요. (자세한 내용이어도 좋습니다.)':'희망진로관련프로그램3', ##2022 추가 질문
                      'S111) 문헌정보학과 교육과정에서 가장 흥미있는 주제는 무엇이었나요? ':'문헌정보학흥미주제3',
                      'S112) 재학 당시 문헌정보학과에 추가로 개설되었다면 좋았을 것 같은 교육과정에 대해서 서술해주세요.':'개설희망과정3',
                      'S131) 문헌정보학과에서 취업을 위한 정보 공유가 적절히 이루어졌다고 생각하시나요?':'학과취업정보교류정도3',
                      'S132) 취업 관련 정보를 얻었던 방법은 무엇인가요? (복수 응답 가능) ':'취업정보습득방법3', ##2022 추가 질문
                      'S133) 재학생들과 졸업생들이 정보 교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수응답 가능)':'학과정보교류선호방법3',
                      'S134) 선호하는 취업정보 공유 방법은 무엇인가요? (복수 응답 가능)':'선호취업정보공유방법3' ##2022 추가 질문
                      }, inplace=True)

In [ ]:
# 구분을 위한 인덱스 열 추가
LIS_2022['index'] = LIS_2022.index
LIS_2022.reset_index(drop=True, inplace=True)
cols = LIS_2022.columns.tolist()
cols.insert(0, cols.pop(cols.index('index')))

In [ ]:
## 학부 재학생 데이터 프레임
Ustudent_2022=LIS_2022[(LIS_2022['졸업여부']=='재학생 (대학원생 포함)')&(LIS_2022['대학원생 여부']=='아니오')]
Ustudent_2022=Ustudent_2022.loc[:,:'대학원진학미고려_이유']   #---수정
Ustudent_2022 = Ustudent_2022.astype({'입학연도':'str'})

## 대학원 재학생 데이터 프레임
Gstudent_2022=LIS_2022[(LIS_2022['졸업여부']=='재학생 (대학원생 포함)')&(LIS_2022['대학원생 여부']=='예 (2022년 입학예정 포함)')]
Gstudent_2022=Gstudent_2022.loc[:,:'대학원진학미고려_이유']
Gstudent_2022 = Gstudent_2022.astype({'입학연도':'str'})

## 재학생(학부+대학원) 데이터 프레임 --- 추가
student_2022 = LIS_2022[LIS_2022['졸업여부']=='재학생 (대학원생 포함)']

## 취업자 데이터 프레임
graduate2022=LIS_2022[LIS_2022['졸업여부']=='졸업생']
temp=graduate2022[['index']]
worker_2022=graduate2022.loc['':,'졸업연도':'향후대학원진학의향_종류2']
worker_2022=pd.concat([temp, worker_2022],axis=1)
worker_2022=worker_2022[worker_2022['취직여부']=='예']
worker_2022.reset_index(drop=True,inplace=True)
worker_2022 = worker_2022.astype({'졸업연도':'str'})

## 취준생 데이터 프레임
temp1=graduate2022.loc[:,'졸업연도']
temp2=graduate2022.loc[:,'문헌정보학흥미주제3':'향후대학원진학의향_종류3']
temp1=pd.DataFrame(temp1)
seeker_2022=pd.concat([temp,temp1,temp2],axis=1)
seeker_2022.dropna(subset = ['희망근무분야3'],inplace=True) #희망근무분야 nan = 취업자임! 그래서 이거 기준으로 드롭함
seeker_2022.reset_index(drop=True,inplace=True)
seeker_2022 = seeker_2022.astype({'졸업연도':'str'})

## 4) 2023 데이터

In [ ]:
# 불필요한 column 정리
LIS_2023origin = LIS_2023origin.drop('타임스탬프', axis=1)
LIS_2023origin = LIS_2023origin.drop('개인 정보를 제외한 설문 분석 결과가 전체 공개되는 것에 동의하십니까?', axis=1)
LIS_2023origin = LIS_2023origin.drop('점수', axis=1)
LIS_2023origin = LIS_2023origin.drop('U213)  현재 희망하는 진로는 무엇인가요?', axis=1)
LIS_2023origin = LIS_2023origin.drop('W213) 현재 근무하고 있는 분야가 무엇인가요?', axis=1)
LIS_2023origin = LIS_2023origin.drop('S213) 현재 희망하는 진로는 무엇인가요?', axis=1)

# 불필요한 column 정리
LIS_2023add = LIS_2023add.drop('Unnamed: 0', axis=1)
LIS_2023add = LIS_2023add.drop('그룹', axis=1)
LIS_2023add = LIS_2023add.drop('희망근무분야_종합', axis=1)

# 데이터 합치기
LIS_2023 = pd.concat([LIS_2023origin, LIS_2023add],axis=1)

In [ ]:
LIS_2023.rename(columns = {
# 인적사항
 '성별은 어떻게 되시나요?' : '성별',
 '졸업 여부는 어떻게 되시나요?' : '졸업여부',
 '입학연도가 어떻게 되시나요? (학번 앞 네자리)' : '입학연도',
 '현재 몇 학년이신가요?' : '학년', #-----2023추가
 '문헌정보가 아닌 다른 주전공을 이수하고 있습니까?' : '주전공',
 '입학연도가 어떻게 되시나요? (학번 앞 네자리).1' : '대학원입학연도',
 '문헌정보가 아닌 다른 주전공을 이수하고 있습니까?.1' : '대학원주전공',

# 재학생 - 개별질문
 'U011) 학부 입학 당시 희망 진로가 있었나요?' : '입학당시희망진로유무',
 'U011.1) 학부 입학 당시 희망 진로는 무엇이었나요? (복수 응답 가능)' : '입학당시희망진로',
 'U012) 현재 희망 진로가 있나요?' : '현재희망진로유무',
 'U012.1) 현재 희망 진로는 무엇인가요? (복수 응답 가능)' : '현재희망진로',
 'U012.2) 현재 희망 진로가 없다면 그 이유는 무엇인가요? (복수 응답 가능)' : '현재희망진로없는이유',
 'U013) 학부 입학 당시와 현재의 희망진로가 다르다면, 그 이유가 무엇인가요?  ' : '희망진로불일치이유',
 'U014) 학부 입학 당시와 현재의 희망진로가 달라진 것에 영향을 준 것은 무엇인가요?' : '희망진로불일치영향',
 'U021) 평소 진로에 있어서 고민이 있다면 무엇인가요? (복수 응답 가능)' : '평소진로고민',
 'U022) 평소 진로 관련 고민을 어떤 방식으로 해결하나요? (복수 응답 가능)' : '진로고민해결방법',
# 재학생 - 학과 관련 질문
 'U111) 문헌정보학과 교육과정에서 가장 흥미있는 주제는 무엇인가요?' : 'U문헌정보학흥미주제',
 'U112) 문헌정보학과에 추가로 개설 되었으면 하는 교육과정이 있다면 서술해주세요.' : 'U추가개설희망과정_주관식',
 'U121) 현재 이수하고 있는(재학 당시 이수한) 다전공이 있나요?' : 'U다전공이수여부',
 'U121.1) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?' : 'U다전공이수종류',
 'U121.2) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?' : 'U다전공단과대', #-----2023추가
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?' : 'U다전공학과1',
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?.1' : 'U다전공학과2',
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?.2' : 'U다전공학과3',
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?.3' : 'U다전공학과4',
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?.4' : 'U다전공학과5',
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?.5' : 'U연계전공학과',
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?.6' : 'U융합전공학과',
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?.7' : 'U자기설계전공',
 'U121.3) 현재 이수하고 있는(재학 당시 이수한) 다전공은 무엇인가요?.8' : 'U2개이상다전공',
 'U121.4) 해당 다전공을 선택한 이유는 무엇인가요?' : 'U다전공선택이유',
 'U121.5) 다전공을 이수하면서 겪은 어려움 혹은 문제가 있다면 무엇인가요? (복수 응답 가능)' : 'U다전공이수어려움',
 'U121.6) 다전공을 이수하지 않은 이유가 무엇인가요?' : 'U다전공미이수이유',
 'U122) 다전공 이수의 장점은 무엇이라고 생각하시나요? (복수 응답 가능)' : 'U다전공이수장점',
 'U123) 다전공 이수의 단점은 무엇이라고 생각하시나요? (복수 응답 가능)' : 'U다전공이수단점',
 'U124) 취직에 다전공 이수가 도움이 된다고 생각하시나요?' : 'U다전공취직도움', #-----2023추가
 'U131) 현재 문헌정보학과에서 취업을 위한 정보 공유가 적절히 이루어지고 있다고 생각하시나요?' : 'U학과취업정보교류정도',
 'U132) 평소 취업 관련 정보를 얻는 방법은 무엇인가요? (복수 응답 가능)' : 'U취업정보수집방법',
 'U133) 선호하는 취업 정보 공유 방법은 무엇인가요? (복수 응답 가능)' : 'U선호취업정보공유방법',
 'U134) 재학생들과 졸업생들이 정보 교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수 응답 가능)' : 'U학과정보교류선호방법',
 'U135) 문헌정보학과에 진로 관련 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하시나요? (복수 응답 가능)' : 'U필요진로관련프로그램',
 'U136) 진로 관련 프로그램과 관련하여 앞의 선지에 언급되지 않은 희망 프로그램을 자유롭게 서술해주세요. (자세한 내용이어도 좋습니다.)' : 'U필요진로관련프로그램_주관식',
 'U137) 문헌정보학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하나요? (복수 응답 가능)' : 'U필요취업강화프로그램',
# 재학생 - 취업 관련 질문
 'U211) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하나요? (복수 응답 가능)' : 'U가능진로종류',
 'U212) 문헌정보학 전공자가 가질 수 있는 진로에 대해서 앞의 선지에 언급되지 않은 내용을 자유롭게 서술해 주세요. (자세한 내용이어도 좋습니다.)' : 'U가능진로종류_주관식',
 'U213.1) 근무하고 싶은 도서관 유형은 무엇인가요? (복수 응답 가능)' : 'U희망근무도서관유형',
 'U213.2) 희망 직종은 무엇인가요? (복수 응답 가능)' : 'U희망직무내용',
 'U213.3) 희망 직무 계열은 무엇인가요?' : 'U희망직무계열',
 'U221) 희망 연봉 수준은 어느 정도인가요?' : 'U희망연봉수준',
 'U231) 희망 진로와 취업을 위해 현재 준비하고 있는 일이 있나요?' : 'U현재준비유무',
 'U231.1) 희망 진로를 위해 현재 준비하고 있는 일이 무엇인가요? (복수 응답 가능)' : 'U현재준비내용',
 'U231.3) 현재 준비하고 있는 일이 없다면 그 이유는 무엇인가요? (복수 응답 가능)' : 'U준비하지않는이유',
 'U232) 취업 준비 중에 가장 힘들 것 같은 것은 무엇인가요? (복수 응답 가능)' : 'U취업준비어려운점',  #-----2023추가
 'U241) 직업을 선택할 때 중요하게 생각하는 가치는 무엇인가요? [1순위]' : 'U직업가치1순위',
 'U241) 직업을 선택할 때 중요하게 생각하는 가치는 무엇인가요? [2순위]' : 'U직업가치2순위',
 'U241) 직업을 선택할 때 중요하게 생각하는 가치는 무엇인가요? [3순위]' : 'U직업가치3순위',
# 재학생 - 대학원 관련 질문
 'U311)  대학원 진학을 고려하고 있나요?' : 'U대학원진학희망여부',
 'U311.1) 어떤 대학원에 진학하고 싶나요?(진학하셨나요?)' : 'U희망진학대학원종류',
 'U311.1) 어떤 세부 전공을 선택하고 싶나요?(선택하셨나요?)' : 'U희망대학원세부전공',  #-----2023추가
 'U311.1) 어떤 대학원에 진학하고 싶나요?(진학하셨나요?).1' : 'U희망진학대학원종류_주관식', #-----2023추가
 'U311.2) 희망(실제) 대학원 진학 시기는 언제인가요?' : 'U희망대학원진학시기',
 'U311.3) 대학원 진학의 목적은 무엇인가요?(무엇이었나요?) (복수 응답 가능)' : 'U대학원진학목적',
 'U311.4) 대학원 진학 준비 중 어려울 것 같은(어려웠던) 점은 무엇인가요? (복수 응답 가능)' : 'U대학원진학어려움',
 'U311.5) 대학원 진학을 고려하지 않는 이유는 무엇이었나요? (복수 응답 가능)' : 'U대학원진학미고려이유',

# 졸업생
 '졸업연도가 어떻게 되시나요? (학부 기준)' : '졸업연도',
 '현재 취직을 하셨나요?' : '취직여부',

# 취직자 - 학과 관련 질문
 'W111) 문헌정보학과 교육과정에서 가장 흥미있는 주제는 무엇이었나요?' : 'W문헌정보학흥미주제',
 'W112) 재학 당시 문헌정보학과에 추가로 개설되었다면 좋았을 것 같은 교육과정에 대해서 서술해주세요.' : 'W추가개설희망과정_주관식',
 'W113) 졸업 후에 도움이 되었던 교육과정은 무엇이었나요? (취업준비/실무 등)' : 'W졸업후도움이된과정',  #-----2023추가
 'W121) 재학 당시 이수한 다전공이 있나요?' : 'W다전공이수여부',
 'W121.1) 재학 당시 이수한 다전공은 무엇인가요?' : 'W다전공이수종류',
 'W121.2) 재학 당시 이수한 다전공은 무엇인가요?' : 'W다전공단과대',  #-----2023추가
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?' : 'W다전공학과1',
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?.1' : 'W다전공학과2',
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?.2' : 'W다전공학과3',
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?.3' : 'W다전공학과4',
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?.4' : 'W다전공학과5',
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?.5' : 'W연계전공학과',
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?.6' : 'W융합전공학과',
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?.7' : 'W자기설계전공',
 'W121.3) 재학 당시 이수한 다전공은 무엇인가요?.8' : 'W2개이상다전공',
 'W121.4) 해당 다전공을 선택한 이유는 무엇인가요?' : 'W다전공선택이유',
 'W121.5) 다전공을 이수하면서 겪은 어려움 혹은 문제가 있었다면 무엇인가요? (복수 응답 가능)' : 'W다전공이수어려움',
 'W121.6) 재학 당시 다전공을 이수하지 않은 이유가 무엇인가요?' : 'W다전공미이수이유',
 'W122) 다전공 이수의 장점은 무엇이라고 생각하시나요? (복수 응답 가능)' : 'W다전공이수장점',
 'W123) 다전공 이수의 단점은 무엇이라고 생각하시나요? (복수 응답 가능)' : 'W다전공이수단점',
 'W124) 취직에 다전공 이수가 도움이 된다고 생각하시나요?' : 'W다전공취직도움',
 'W125) 취직에 다전공 이수가 도움이 된다고 생각하시는 이유를 서술해주세요.' : 'W다전공취직도움이유_주관식',   #-----2023추가
 'W131) 문헌정보학과에서 취업을 위한 정보 공유가 적절히 이루어졌다고 생각하시나요?' : 'W학과취업정보교류정도',
 'W132) 취업 관련 정보를 얻었던 방법은 무엇인가요? (복수 응답 가능)' : 'W취업정보수집방법',
 'W133) 선호하는 취업 정보 공유 방법은 무엇인가요? (복수 응답 가능)' : 'W선호취업정보공유방법',
 'W134) 재학생들과 졸업생들이 정보 교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수 응답 가능)' : 'W학과정보교류선호방법',
 'W135) 문헌정보학과에 진로 관련 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하시나요? (복수 응답 가능)' : 'W필요진로관련프로그램',
 'W136) 진로 관련 프로그램과 관련하여 앞의 선지에 언급되지 않은 희망 프로그램을 자유롭게 서술해주세요. (자세한 내용이어도 좋습니다.)' : 'W필요진로관련프로그램_주관식',
 'W137) 문헌정보학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하나요? (복수 응답 가능)' : 'W필요취업강화프로그램',
# 취직자 - 취업 관련 질문
 'W211) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하나요?' : 'W가능진로종류',
 'W212) 문헌정보학 전공자가 가질 수 있는 진로에 대해서 앞의 선지에 언급되지 않은 내용을 자유롭게 서술해 주세요. (자세한 내용이어도 좋습니다.)' : 'W가능진로종류_주관식',
 'W213.1) 근무하고 있는 도서관 유형은 무엇인가요? (복수 응답 가능)' : 'W근무도서관유형',
 'W213.2) 현재 근무 중인 직종은 무엇인가요? (복수 응답 가능)' : 'W현재직무내용',
 'W213.3) 현재 근무 중인 직무 계열은 무엇인가요?' : 'W현재직무계열',
 'W213.4) 현재 근무 중인 분야에 대해 최대한 자세히 작성해주세요.' : 'W현재근무분야_주관식',
 'W221) 실제 연봉 수준은 어느 정도인가요?' : 'W실제연봉수준',
 'W231) 희망 진로와 취업을 위해 과거에 준비했던 일이 있나요?' : 'W과거준비유무',
 'W231.1) 희망 진로와 취업을 위해 준비했던 일이 무엇인가요? (복수 응답 가능)' : 'W과거준비내용',
 'W231.2) 준비했던 일 중 가장 도움이 되었던 것은 무엇인가요?' : 'W가장도움이되었던준비',  #-----2023추가
 'W231.3) 과거에 준비하고 있던 일이 없었다면 그 이유는 무엇인가요? (복수 응답 가능)' : 'W준비하지않은이유',
 'W232) 취업 준비 중에 가장 힘들었던 것은 무엇인가요? (복수 응답 가능)' : 'W취업준비어려운점',
 'W233) 실제 취업 준비 기간은 어느 정도이었나요?' : 'W취업준비기간',
 'W241) 직업을 선택할 때 중요하게 생각했던 가치는 무엇인가요?  [1순위]' : 'W직업가치1순위',
 'W241) 직업을 선택할 때 중요하게 생각했던 가치는 무엇인가요?  [2순위]' : 'W직업가치2순위',
 'W241) 직업을 선택할 때 중요하게 생각했던 가치는 무엇인가요?  [3순위]' : 'W직업가치3순위',
 'W242) 취직 전후로 직업 가치관의 변화가 있었나요?' : 'W취직전후가치관변화',
 'W242.1) 취직 후 중요시하게 된 가치는 무엇인가요? [1순위]' : 'W취직후가치1순위',
 'W242.1) 취직 후 중요시하게 된 가치는 무엇인가요? [2순위]' : 'W취직후가치2순위',
 'W242.1) 취직 후 중요시하게 된 가치는 무엇인가요? [3순위]' : 'W취직후가치3순위',
 'W251) 현재 직무 만족도는 어떠한가요?' : 'W현재직무만족도',
 'W252) 현재의 근무지를 문헌정보학과 후배들에게 추천할 의향이 있나요?' : 'W추천의향',
# 취직자 - 대학원 관련 질문
 'W311) 대학원에 진학했나요?' : 'W대학원진학여부',
 'W311.1) 어떤 대학원에 진학하셨나요?' : 'W진학대학원종류',
 'W311.1) 어떤 세부 전공을 선택하셨나요?' : 'W대학원세부전공',  #-----2023추가
 'W311.1) 어떤 대학원에 진학하셨나요?.1' : 'W진학대학원종류_주관식', #-----2023추가
 'W311.2) 대학원 진학 시기는 언제인가요?' : 'W대학원진학시기',
 'W311.3) 대학원 진학의 목적은 무엇이었나요? (복수 응답 가능)' : 'W대학원진학목적',
 'W311.4) 대학원 진학 준비 중 어려웠던 점은 무엇인가요? (복수 응답 가능)' : 'W대학원진학어려움',
 'W311.5) 대학원 진학을 고려하지 않은 이유는 무엇이었나요? (복수 응답 가능)' : 'W대학원진학미고려이유',
 'W311.6) 향후 대학원에 진학할 의향이 있으신가요?' : 'W진학의향여부',
 'W311.7) 진학할 의향이 있다면 어떤 대학원에 진학할 생각이신가요? (복수 응답 가능)' : 'W향후대학원종류',

# 취준생 - 학과 관련 질문
 'S111) 문헌정보학과 교육과정에서 가장 흥미있는 주제는 무엇이었나요?' : 'S문헌정보학흥미주제',
 'S112) 재학 당시 문헌정보학과에 추가로 개설되었다면 좋았을 것 같은 교육과정에 대해서 서술해주세요.' : 'S추가개설희망과정_주관식',
 'S113) 졸업 후에 도움이 되었던 교육과정은 무엇이었나요? (취업준비 등)' : 'S졸업후도움이된과정',  #-----2023추가
 'S121) 재학 당시 이수한 다전공이 있나요?' : 'S다전공이수여부',
 'S121.1) 재학 당시 이수한 다전공은 무엇인가요?' : 'S다전공이수종류',
 'S121.2) 재학 당시 이수한 다전공은 무엇인가요?' : 'S다전공단과대',  #-----2023추가
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?' : 'S다전공학과1',
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?.1' : 'S다전공학과2',
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?.2' : 'S다전공학과3',
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?.3' : 'S다전공학과4',
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?.4' : 'S다전공학과5',
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?.5' : 'S연계전공학과',
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?.6' : 'S융합전공학과',
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?.7' : 'S자기설계전공',
 'S121.3) 재학 당시 이수한 다전공은 무엇인가요?.8' : 'S2개이상다전공',
 'S121.4) 해당 다전공을 선택한 이유는 무엇인가요?' : 'S다전공선택이유',
 'S121.5 다전공을 이수하면서 겪은 어려움 혹은 문제가 있다면 무엇인가요? (복수 응답 가능)' : 'S다전공이수어려움',
 'S121.6) 재학 당시 다전공을 이수하지 않은 이유가 무엇인가요?' : 'S다전공미이수이유',
 'S122) 다전공 이수의 장점은 무엇이라고 생각하시나요? (복수 응답 가능)' : 'S다전공이수장점',
 'S123) 다전공 이수의 단점은 무엇이라고 생각하시나요? (복수 응답 가능)' : 'S다전공이수단점',
 'S124)  취직 준비에 다전공 이수가 도움이 된다고 생각하시나요?' : 'S다전공취직도움', #-----2023추가
 'S131) 문헌정보학과에서 취업을 위한 정보 공유가 적절히 이루어졌다고 생각하시나요?' : 'S학과취업정보교류정도',
 'S132) 평소 취업 관련 정보를 얻는 방법은 무엇인가요? (복수 응답 가능)' : 'S취업정보수집방법',
 'S133) 선호하는 취업 정보 공유 방법은 무엇인가요? (복수 응답 가능)' : 'S선호취업정보공유방법',
 'S134) 재학생들과 졸업생들이 정보 교류를 하기에 어떤 방법이 적절하다고 생각하시나요? (복수응답 가능)' : 'S학과정보교류선호방법',
 'S135) 문헌정보학과에 진로 관련 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하시나요? (복수 응답 가능)' : 'S필요진로관련프로그램',
 'S136) 진로 관련 프로그램과 관련하여 앞의 선지에 언급되지 않은 희망 프로그램을 자유롭게 서술해주세요. (자세한 내용이어도 좋습니다.)' : 'S필요진로관련프로그램_주관식',
 'S137) 문헌정보학과에 취업 강화 프로그램이 필요하다면, 어떤 것이 필요하다고 생각하나요? (복수 응답 가능)' : 'S필요취업강화프로그램',
 # 취준생 - 취업 관련 질문
 'S211) 문헌정보학 전공자가 가질 수 있는 진로에는 어떤 것이 있다고 생각하나요?' : 'S가능진로종류',
 'S212) 문헌정보학 전공자가 가질 수 있는 진로에 대해서 앞의 선지에 언급되지 않은 내용을 자유롭게 서술해 주세요. (자세한 내용이어도 좋습니다.)' : 'S가능진로종류_주관식',
 'S213.1) 근무하고 싶은 도서관 유형은 무엇인가요? (복수 응답 가능)' : 'S희망근무도서관유형',
 'S213.2) 희망 직종은 무엇인가요? (복수 응답 가능)' : 'S희망직무내용',
 'S213.3) 희망 직무 계열은 무엇인가요?' : 'S희망직무계열',
 'S221) 희망 연봉 수준은 어느 정도인가요?' : 'S희망연봉수준',
 'S231) 희망 진로와 취업을 위해 현재 준비하고 있는 일이 있나요?' : 'S현재준비유무',
 'S231.1) 희망 진로와 취업을 위해 현재 준비하고 있는 일이 무엇인가요? (복수 응답 가능)' : 'S현재준비내용',
 'S231.3) 현재 준비하고 있는 일이 없다면 그 이유는 무엇인가요? (복수 응답 가능)' : 'S준비하지않는이유',
 'S232) 취업 준비 중에 가장 힘든 것은 무엇인가요? (복수 응답 가능)' : 'S취업준비어려운점',
 'S241) 직업을 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [1순위]' : 'S직업가치1순위',
 'S241) 직업을 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [2순위]' : 'S직업가치2순위',
 'S241) 직업을 선택할 때 중요하게 생각하는 가치는 무엇인가요?  [3순위]' : 'S직업가치3순위',
# 취준생 - 대학원 관련 질문
 'S311) 대학원에 진학했나요?' : 'S대학원진학여부',
 'S311.1) 어떤 대학원에 진학하셨나요?' : 'S진학대학원종류',
 'S311.1) 어떤 세부 전공을 선택하셨나요?' : 'S대학원세부전공',  #-----2023추가
 'S311.1) 어떤 대학원에 진학하셨나요?.1' : 'S진학대학원종류_주관식', #-----2023추가
 'S311.2) 대학원 진학 시기는 언제인가요?' : 'S대학원진학시기',
 'S311.3) 대학원 진학의 목적은 무엇이었나요? (복수 응답 가능)' : 'S대학원진학목적',
 'S311.4) 대학원 진학 준비 중 어려웠던 점은 무엇인가요? (복수 응답 가능)' : 'S대학원진학어려움',
 'S311.5) 대학원 진학을 고려하지 않은 이유는 무엇이었나요? (복수 응답 가능)' : 'S대학원진학미고려이유',
 'S311.6) 향후 대학원에 진학할 의향이 있으신가요?' : 'S진학의향여부',
 'S311.7) 진학할 의향이 있다면 어떤 대학원에 진학할 생각이신가요? (복수 응답 가능)' : 'S향후대학원종류'
}, inplace=True)

In [ ]:
cols1 = LIS_2023.columns.tolist()
cols1.insert(LIS_2023.columns.get_loc('U희망근무도서관유형'), cols1.pop(cols1.index('U희망근무분야')))
LIS_2023 = LIS_2023[cols1]
cols2 = LIS_2023.columns.tolist()
cols2.insert(LIS_2023.columns.get_loc('W근무도서관유형'), cols2.pop(cols2.index('W근무분야')))
LIS_2023 = LIS_2023[cols2]
cols3 = LIS_2023.columns.tolist()
cols3.insert(LIS_2023.columns.get_loc('S희망근무도서관유형'), cols3.pop(cols3.index('S희망근무분야')))
LIS_2023 = LIS_2023[cols3]

In [ ]:
# 구분을 위한 인덱스 열 추가
LIS_2023['index'] = LIS_2023.index
LIS_2023.reset_index(drop=True, inplace=True)
cols7 = LIS_2023.columns.tolist()
cols7.insert(0, cols7.pop(cols7.index('index')))
LIS_2023 = LIS_2023[cols7]
# 연도 자료형 변경
cols8 = ['입학연도', '대학원입학연도', '졸업연도']
LIS_2023[cols8] = LIS_2023[cols8].apply(lambda x: x.dropna().astype(int).astype(str))
# 3개년 설문과 비교하기 위해 선택지 통일(‘자영업’ 응답을 ’기타‘ 처리함)
cols9 = ['U희망근무분야', 'S희망근무분야', 'W근무분야']
LIS_2023[cols9] = LIS_2023[cols9].replace('자영업', '기타')
# 표기 오류 수정
LIS_2023['U희망연봉수준']=LIS_2023['U희망연봉수준'].replace('2000만원 이상 2500만원 이하', '2000만원 이상 2500만원 미만')
LIS_2023['S희망연봉수준']=LIS_2023['S희망연봉수준'].replace('2000만원 이상 2500만원 이하', '2000만원 이상 2500만원 미만')
LIS_2023['W희망연봉수준']=LIS_2023['W실제연봉수준'].replace('2000만원 이상 2500만원 이하', '2000만원 이상 2500만원 미만')

In [ ]:
## 학부 재학생 데이터 프레임
Ustudent_2023 = LIS_2023[LIS_2023['졸업여부'] == '학부 재학생']
Uindex1 = LIS_2023.columns.get_loc('주전공')
Uindex2 = LIS_2023.columns.get_loc('입학당시희망진로유무')
Uindex3 = LIS_2023.columns.get_loc('U대학원진학미고려이유')
Ustudent_2023  = Ustudent_2023 .iloc[:, list(range(0, Uindex1+1)) + list(range(Uindex2, Uindex3+1))]
Ustudent_2023 .reset_index(drop=True,inplace=True)

## 대학원 재학생 데이터 프레임
Gstudent_2023 = LIS_2023[LIS_2023['졸업여부'] == '대학원 재학생']
Gindex1 = LIS_2023.columns.get_loc('졸업여부')
Gindex2 = LIS_2023.columns.get_loc('대학원입학연도')
Gindex3 = LIS_2023.columns.get_loc('U대학원진학미고려이유')
Gstudent_2023 = Gstudent_2023.iloc[:, list(range(0, Gindex1+1)) + list(range(Gindex2, Gindex3+1))]
Gstudent_2023.reset_index(drop=True,inplace=True)

## 재학생(학부+대학원) 데이터 프레임 --- 추가
student_2023 = pd.concat([Ustudent_2023 ,Gstudent_2023],ignore_index=True)

## 취준생 데이터 프레임
seeker_2023 = LIS_2023[LIS_2023['취직여부'] == '아니오']
Sindex1 = LIS_2023.columns.get_loc('졸업여부')
Sindex2 = LIS_2023.columns.get_loc('졸업연도')
Sindex3 = LIS_2023.columns.get_loc('취직여부')
Sindex4 = LIS_2023.columns.get_loc('S문헌정보학흥미주제')
Sindex5 = len(LIS_2023.columns)
seeker_2023 = seeker_2023.iloc[:, list(range(0, Sindex1+1)) + list(range(Sindex2, Sindex3+1)) + list(range(Sindex4, Sindex5))]
seeker_2023.reset_index(drop=True,inplace=True)

## 취업자 데이터 프레임
worker_2023 = LIS_2023[LIS_2023['취직여부'] == '예']
Windex1 = LIS_2023.columns.get_loc('졸업여부')
Windex2 = LIS_2023.columns.get_loc('졸업연도')
Windex3 = LIS_2023.columns.get_loc('W향후대학원종류')
worker_2023 = worker_2023.iloc[:, list(range(0, Windex1+1)) + list(range(Windex2, Windex3+1))]
worker_2023.reset_index(drop=True,inplace=True)

## 5) 공통함수 지정

In [ ]:
# 답변 정리 후 데이터 프레임 만들기
def count_to(df):

    # 시리즈 복사 - 원본 데이터 유지
    df_copy = df.copy()
    # 결측값 제거
    df_copy.dropna(inplace=True)
    # 시리즈를 리스트로 변환
    first_list = df_copy.tolist()

    # 쪼개야 할 단어들을 찾아서 리스트에서 제거하고 각 단어를 쪼갠 뒤 리스트에 추가
    words_to_split = [word for word in first_list if ',' in str(word) or ';' in str(word)]
    for word in words_to_split:
      first_list.remove(word)
      first_list.extend(re.split(r'[;,]', str(word)))

    # 중복을 제거한 고유한 단어들의 리스트 생성
    unique_words = list(set(first_list))
    # 각 고유한 단어의 출현 빈도 리스트에 저장
    counts = [first_list.count(word) for word in unique_words]

    # 단어와 출현 빈도로 데이터프레임 생성
    count_df = pd.DataFrame({df.name: unique_words, '카운트': counts})
    # 출현 빈도에 따라 내림차순으로 데이터프레임 정렬
    count_df = count_df.sort_values('카운트', ascending=False).reset_index(drop=True)

    # 정렬된 데이터프레임 반환
    return count_df

In [ ]:
# 예외처리 함수
def count_to_outlier(df, column):
    # 공백을 제거한 리스트 생성
    tmp_list = [i.strip() for i in df[column]]
    # 데이터프레임의 열을 업데이트
    df[column] = tmp_list
    # 공백을 제거한 값에 기반하여 그룹화하고 합계 계산
    df = df.groupby(column).sum()
    # 데이터프레임 정렬 후 인덱스 초기화
    df = df.sort_values('카운트', ascending=True)
    df.reset_index(inplace=True)

    return df

In [ ]:
# 구분/퍼센트 추가 함수
def add_classes(df, col, ex_column, new_column):
    # 데이터프레임의 인덱스 재설정
    df.reset_index(drop=True, inplace=True)
    # '구분' 열에 'col' 값을 할당
    df['구분'] = col
    per = []
    # 각 행에 대해 '카운트'의 비율을 계산하여 '퍼센트(%)' 열에 추가
    for i in range(0, len(df)):
        per.append(round(df['카운트'].loc[i] / df['카운트'].sum() * 100, 2))
    df['퍼센트(%)'] = per
    # 열 이름 변경
    df.rename(columns={ex_column: new_column}, inplace=True)
    # '퍼센트(%)' 열을 기준으로 내림차순 정렬 후 인덱스 초기화
    df.sort_values(by='퍼센트(%)', ascending=False, inplace=True)
    df.reset_index(drop = True, inplace=True)
    # 데이터 프레임 컬럼 순서 변경
    df = df[['구분', new_column, '카운트', '퍼센트(%)']]
    return df

In [ ]:
# 기타 처리 함수
def others(df, col, values_to_replace): #values_to_replace는 리스트 형태로 작성
    df[col] = df[col].replace(values_to_replace, '기타')
    df.loc[df[col] == '기타', '카운트'] = df.loc[df[col] == '기타']['카운트'].sum()
    df = df.drop_duplicates(subset=col, keep='first')
    return df

In [ ]:
# 연도 기준 정렬 함수
def sorted_df_year(df, col):
  # 기타 정렬을 위해 이름 변경 (기타 항목 있는 경우)
  df[col] = df[col].apply(lambda x: '힣' if x == '기타' else x)
  df[col] = df[col].apply(lambda x: '힣힣' if x == '없음' else x)
  # 연도 정렬 기준 고정 후 컬럼 오름차순 정렬
  categories = ['2021', '2022', '2023']
  df['연도'] = pd.Categorical(df['연도'], categories=categories, ordered=True)
  df = df.sort_values(by=['연도', col])
  # 기타 정렬 후 이름 변경 (기타 항목 있는 경우)
  df[col] = df[col].apply(lambda x: '기타' if x == '힣' else x)
  df[col] = df[col].apply(lambda x: '없음' if x == '힣힣' else x)
  # 인덱스 초기화
  df.reset_index(drop=True,inplace=True)
  return df

# 2️⃣ 비교 분석

## **입학 당시와 현재 희망 진로 종류 비교** (학부 재학생)

* **2021**

In [ ]:
DF_2021_prefer1 = count_to(Ustudent_2021['희망진로_입학'])
DF_2021_prefer1 = count_to_outlier(DF_2021_prefer1, '희망진로_입학')

# 국회도서관/의학도서관 사서 > 사서로 포함
DF_2021_prefer1 = DF_2021_prefer1[DF_2021_prefer1['희망진로_입학'] != '국회도서관/의학도서관 사서']
DF_2021_prefer1.loc[DF_2021_prefer1['희망진로_입학'] == '사서', '카운트'] += 1

# 영상콘텐츠 제작자 > 기타 처리
DF_2021_prefer1 = others(DF_2021_prefer1, '희망진로_입학', ['영상콘텐츠 제작자'])

DF_2021_prefer1 = add_classes(DF_2021_prefer1,'2021재학생','희망진로_입학','입학당시희망진로')
DF_2021_prefer1

,구분,입학당시희망진로,카운트,퍼센트(%)
0,2021재학생,사서,34,24.29
1,2021재학생,사서 교사,27,19.29
2,2021재학생,공무원,14,10.00
3,2021재학생,데이터 전문가,11,7.86
4,2021재학생,언론인,11,7.86
5,2021재학생,기록물 관리자,9,6.43
6,2021재학생,출판업,9,6.43
7,2021재학생,일반 사기업,8,5.71
8,2021재학생,공기업,7,5.00
9,2021재학생,대학교 (교수 및 직원),5,3.57


In [ ]:
DF_2021_prefer2 = count_to(Ustudent_2021['희망진로_현재'])
DF_2021_prefer2 = count_to_outlier(DF_2021_prefer2, '희망진로_현재')

# 국회도서관/의학도서관 사서 > 사서로 포함
DF_2021_prefer2 = DF_2021_prefer2[DF_2021_prefer2['희망진로_현재'] != '국회도서관/의학도서관 사서']
DF_2021_prefer2.loc[DF_2021_prefer2['희망진로_현재'] == '사서', '카운트'] += 1

# 데이터사이언티스트 > 데이터 전문가로 포함
DF_2021_prefer2 = DF_2021_prefer2[DF_2021_prefer2['희망진로_현재'] != '데이터사이언티스트']
DF_2021_prefer2.loc[DF_2021_prefer2['희망진로_현재'] == '데이터 전문가', '카운트'] += 1

# 군인, 변리사, 성우 > 기타 처리
DF_2021_prefer2 = others(DF_2021_prefer2, '희망진로_현재', ['군인', '변리사', '성우'])

DF_2021_prefer2 = add_classes(DF_2021_prefer2,'2021재학생','희망진로_현재','현재희망진로')
DF_2021_prefer2

,구분,현재희망진로,카운트,퍼센트(%)
0,2021재학생,사서,38,22.89
1,2021재학생,공무원,22,13.25
2,2021재학생,사서 교사,19,11.45
3,2021재학생,데이터 전문가,15,9.04
4,2021재학생,일반 사기업,15,9.04
5,2021재학생,공기업,14,8.43
6,2021재학생,대학교 (교수 및 직원),12,7.23
7,2021재학생,기록물 관리자,10,6.02
8,2021재학생,출판업,9,5.42
9,2021재학생,개발자 (프로그래머),5,3.01


* **2022**

In [ ]:
DF_2022_prefer1 = count_to(Ustudent_2022['희망진로_입학'])
DF_2022_prefer1 = count_to_outlier(DF_2022_prefer1, '희망진로_입학')

# KISTI 연구원 (연구직종), 미디어 직무 > 기타 처리
DF_2022_prefer1 = others(DF_2022_prefer1, '희망진로_입학', ['KISTI 연구원 (연구직종)', '미디어 직무'])

DF_2022_prefer1 = add_classes(DF_2022_prefer1,'2022재학생','희망진로_입학','입학당시희망진로')
DF_2022_prefer1

<ipython-input-22-910dffb5a43a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['구분'] = col
<ipython-input-22-910dffb5a43a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['퍼센트(%)'] = per
<ipython-input-22-910dffb5a43a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={ex_column: new_column}, inp

,구분,입학당시희망진로,카운트,퍼센트(%)
0,2022재학생,사서,35,31.53
1,2022재학생,사서 교사,19,17.12
2,2022재학생,공무원,15,13.51
3,2022재학생,일반 사기업,8,7.21
4,2022재학생,출판업,7,6.31
5,2022재학생,기록물 관리자,6,5.41
6,2022재학생,데이터 전문가,5,4.50
7,2022재학생,공기업,4,3.60
8,2022재학생,대학교 (교수 및 직원),4,3.60
9,2022재학생,개발자 (프로그래머),3,2.70


In [ ]:
DF_2022_prefer2 = count_to(Ustudent_2022['희망진로_현재'])
DF_2022_prefer2 = count_to_outlier(DF_2022_prefer2, '희망진로_현재')

# 의회공무원 > 공무원으로 포함
DF_2022_prefer2 = DF_2022_prefer2[DF_2022_prefer2['희망진로_현재'] != '의회공무원']
DF_2022_prefer2.loc[DF_2022_prefer2['희망진로_현재'] == '공무원', '카운트'] += 1

# 전문직, 학예사 또는 서지학 관련 분야 진출, 미디어 직무, 성우, 세무사, 인권단체 활동가, KISTI 연구원 아직 미정	> 기타 처리
DF_2022_prefer2 = others(DF_2022_prefer2, '희망진로_현재', ['전문직', '학예사 또는 서지학 관련 분야 진출', '미디어 직무', '성우', '세무사', '인권단체 활동가', 'KISTI 연구원 아직 미정'])

DF_2022_prefer2 = add_classes(DF_2022_prefer2,'2022재학생','희망진로_현재','현재희망진로')
DF_2022_prefer2

,구분,현재희망진로,카운트,퍼센트(%)
0,2022재학생,사서,27,21.09
1,2022재학생,공무원,18,14.06
2,2022재학생,일반 사기업,14,10.94
3,2022재학생,데이터 전문가,13,10.16
4,2022재학생,기록물 관리자,9,7.03
5,2022재학생,공기업,9,7.03
6,2022재학생,사서 교사,9,7.03
7,2022재학생,출판업,8,6.25
8,2022재학생,기타,7,5.47
9,2022재학생,대학교 (교수 및 직원),5,3.91


* **2023**

In [ ]:
DF_2023_prefer1 = count_to(Ustudent_2023['입학당시희망진로'])
DF_2023_prefer1 = count_to_outlier(DF_2023_prefer1, '입학당시희망진로')

# 로스쿨, 자영업 > 기타 처리
DF_2023_prefer1 = others(DF_2023_prefer1, '입학당시희망진로', ['로스쿨', '자영업'])

DF_2023_prefer1 = add_classes(DF_2023_prefer1,'2023재학생','입학당시희망진로','입학당시희망진로')
DF_2023_prefer1

,구분,입학당시희망진로,카운트,퍼센트(%)
0,2023재학생,사서직 (공무원/대학 및 전문 도서관 사서 포함),32,33.33
1,2023재학생,사서 교사,22,22.92
2,2023재학생,공기업,8,8.33
3,2023재학생,데이터 전문가,8,8.33
4,2023재학생,기록물 관리사,6,6.25
5,2023재학생,일반 사기업,5,5.21
6,2023재학생,개발자 (프로그래머),3,3.12
7,2023재학생,공무원 (사서직 제외),3,3.12
8,2023재학생,언론인,3,3.12
9,2023재학생,출판업,3,3.12


In [ ]:
DF_2023_prefer2 = count_to(Ustudent_2023['현재희망진로'])
DF_2023_prefer2 = count_to_outlier(DF_2023_prefer2, '현재희망진로')

#승무원, 언론인, 콘텐츠 매니지먼트 창업, 회계사 -> 기타 처리
DF_2023_prefer2 = others(DF_2023_prefer2, '현재희망진로', ['승무원', '언론인', '콘텐츠 매니지먼트 창업', '회계사'])
# 로스쿨, 자영업 > 기타 처리
DF_2023_prefer2 = others(DF_2023_prefer2, '현재희망진로', ['로스쿨', '자영업'])

DF_2023_prefer2 = add_classes(DF_2023_prefer2,'2023재학생','현재희망진로','현재희망진로')
DF_2023_prefer2

,구분,현재희망진로,카운트,퍼센트(%)
0,2023재학생,일반 사기업,23,20.91
1,2023재학생,사서직 (공무원/대학 및 전문 도서관 사서 포함),18,16.36
2,2023재학생,데이터 전문가,16,14.55
3,2023재학생,개발자 (프로그래머),11,10.00
4,2023재학생,기타,9,8.18
5,2023재학생,공기업,9,8.18
6,2023재학생,사서 교사,7,6.36
7,2023재학생,공무원 (사서직 제외),6,5.45
8,2023재학생,기록물 관리사,4,3.64
9,2023재학생,대학 교수,4,3.64


 **입학 당시 희망 진로**

In [ ]:
DF_2021_prefer1['입학당시희망진로'] = DF_2021_prefer1['입학당시희망진로'].replace({
    '기록물 관리자':'기록물 관리사',
    '대학교 (교수 및 직원)':'대학 교수'})
DF_2022_prefer1['입학당시희망진로'] = DF_2022_prefer1['입학당시희망진로'].replace({
    '기록물 관리자':'기록물 관리사',
    '대학교 (교수 및 직원)':'대학 교수'})
DF_2023_prefer1['입학당시희망진로'] = DF_2023_prefer1['입학당시희망진로'].replace({
    '사서직 (공무원/대학 및 전문 도서관 사서 포함)': '사서',
    '공무원 (사서직 제외)':'공무원'})

In [ ]:
DF_prefer1 = pd.concat([DF_2021_prefer1, DF_2022_prefer1, DF_2023_prefer1],axis=0)

In [ ]:
percent_sum = DF_prefer1.groupby('입학당시희망진로')['퍼센트(%)'].sum().reset_index()
DF_prefer1 = pd.merge(DF_prefer1, percent_sum, how='left', on='입학당시희망진로', suffixes=('', '_합계'))
DF_prefer1=DF_prefer1.sort_values(by='퍼센트(%)_합계', ascending = False)

In [ ]:
# 입학 당시 희망진로 비교 그래프 - 선호도 내림차순 정렬

colors = [px.colors.sequential.dense[0], px.colors.sequential.dense[2], px.colors.sequential.dense[5]]

# 그룹화된 막대 차트 생성
fig = px.bar(DF_prefer1, x='입학당시희망진로', y='퍼센트(%)', color='구분',
             color_discrete_sequence=colors, text='퍼센트(%)', barmode='group',
             custom_data=['구분', '카운트', '입학당시희망진로'],
             category_orders={'구분': ["2021재학생", "2022재학생", "2023재학생"]},
             orientation='v')

# 레이아웃 지정
fig.update_layout(bargap=0.15,height=450, width=2000,
    xaxis={'visible': True, 'showticklabels': True, 'title': ''},
    yaxis={'visible': False, 'showticklabels': False},
    showlegend=True,
    plot_bgcolor="#FFFFFF"
)

fig.update_traces(texttemplate='%{text:.f}%', textposition='inside', textfont_size=8)
fig.update_traces(hovertemplate='<br>%{customdata[0]} <br>%{customdata[2]} <br>응답 건수: %{customdata[1]} 건<extra></extra>')

fig.update_xaxes(tickfont=dict(size=11))

# 그래프 출력
fig.show()

# html 변환
fig.write_html("입학당시희망진로3개년.html")

**현재 희망 진로**

In [ ]:
DF_2021_prefer2['현재희망진로'] = DF_2021_prefer2['현재희망진로'].replace({
    '기록물 관리자':'기록물 관리사',
    '대학교 (교수 및 직원)':'대학 교수'})
DF_2022_prefer2['현재희망진로'] = DF_2022_prefer2['현재희망진로'].replace({
    '기록물 관리자':'기록물 관리사',
    '대학교 (교수 및 직원)':'대학 교수'})
DF_2023_prefer2['현재희망진로'] = DF_2023_prefer2['현재희망진로'].replace({
    '사서직 (공무원/대학 및 전문 도서관 사서 포함)': '사서',
    '공무원 (사서직 제외)':'공무원'})

In [ ]:
DF_prefer2 = pd.concat([DF_2021_prefer2, DF_2022_prefer2, DF_2023_prefer2],axis=0)

In [ ]:
percent_sum = DF_prefer2.groupby('현재희망진로')['퍼센트(%)'].sum().reset_index()
DF_prefer2 = pd.merge(DF_prefer2, percent_sum, how='left', on='현재희망진로', suffixes=('', '_합계'))
DF_prefer2=DF_prefer2.sort_values(by='퍼센트(%)_합계', ascending = False)

In [ ]:
# 입학 당시 희망진로 비교 그래프 - 선호도 내림차순 정렬

colors = [px.colors.sequential.dense[0], px.colors.sequential.dense[2], px.colors.sequential.dense[5]]

# 그룹화된 막대 차트 생성
fig = px.bar(DF_prefer2, x='현재희망진로', y='퍼센트(%)', color='구분',
             color_discrete_sequence=colors, text='퍼센트(%)', barmode='group',
             custom_data=['구분', '카운트', '현재희망진로'],
             category_orders={'구분': ["2021재학생", "2022재학생", "2023재학생"]},
             orientation='v')

# 레이아웃 지정
fig.update_layout(bargap=0.15,height=450, width=2000,
    xaxis={'visible': True, 'showticklabels': True, 'title': ''},
    yaxis={'visible': False, 'showticklabels': False},
    showlegend=True,
    plot_bgcolor="#FFFFFF"
)

fig.update_traces(texttemplate='%{text:.f}%', textposition='inside', textfont_size=8)
fig.update_traces(hovertemplate='<br>%{customdata[0]} <br>%{customdata[2]} <br>응답 건수: %{customdata[1]} 건<extra></extra>')

fig.update_xaxes(tickfont=dict(size=11))

# 그래프 출력
fig.show()

# html 변환
fig.write_html("현재희망진로3개년.html")

**입학 당시/현재 비교**

In [ ]:
# 연도별 입학 당시/현재 희망진로 비교 그래프 - 선호도 내림차순
fig = go.Figure(data=[
    go.Bar(name='2021입학당시', x=DF_prefer1[DF_prefer1['구분']=='2021재학생']['입학당시희망진로'], y=DF_prefer1[DF_prefer1['구분']=='2021재학생']['퍼센트(%)'], marker=dict(color=px.colors.sequential.dense[0]), hovertemplate='응답 건수: %{customdata}건<extra></extra>', customdata=DF_prefer1[DF_prefer1['구분']=='2021재학생']['카운트'], text=DF_prefer1[DF_prefer1['구분']=='2021재학생']['퍼센트(%)']),
    go.Bar(name='2021현재', x=DF_prefer2[DF_prefer2['구분']=='2021재학생']['현재희망진로'], y=DF_prefer2[DF_prefer2['구분']=='2021재학생']['퍼센트(%)'], marker=dict(color=px.colors.sequential.dense[0]), hovertemplate='응답 건수: %{customdata}건<extra></extra>', customdata=DF_prefer2[DF_prefer2['구분']=='2021재학생']['카운트'], text=DF_prefer2[DF_prefer2['구분']=='2021재학생']['퍼센트(%)']),
    go.Bar(name='2022입학당시', x=DF_prefer1[DF_prefer1['구분']=='2022재학생']['입학당시희망진로'], y=DF_prefer1[DF_prefer1['구분']=='2022재학생']['퍼센트(%)'], marker=dict(color=px.colors.sequential.dense[2]), hovertemplate='응답 건수: %{customdata}건<extra></extra>', customdata=DF_prefer1[DF_prefer1['구분']=='2022재학생']['카운트'], text=DF_prefer1[DF_prefer1['구분']=='2022재학생']['퍼센트(%)']),
    go.Bar(name='2022현재', x=DF_prefer2[DF_prefer2['구분']=='2022재학생']['현재희망진로'], y=DF_prefer2[DF_prefer2['구분']=='2022재학생']['퍼센트(%)'], marker=dict(color=px.colors.sequential.dense[2]), hovertemplate='응답 건수: %{customdata}건<extra></extra>', customdata=DF_prefer2[DF_prefer2['구분']=='2022재학생']['카운트'], text=DF_prefer2[DF_prefer2['구분']=='2022재학생']['퍼센트(%)']),
    go.Bar(name='2023입학당시', x=DF_prefer1[DF_prefer1['구분']=='2023재학생']['입학당시희망진로'], y=DF_prefer1[DF_prefer1['구분']=='2023재학생']['퍼센트(%)'], marker=dict(color=px.colors.sequential.dense[5]), hovertemplate='응답 건수: %{customdata}건<extra></extra>', customdata=DF_prefer1[DF_prefer1['구분']=='2023재학생']['카운트'], text=DF_prefer1[DF_prefer1['구분']=='2023재학생']['퍼센트(%)']),
    go.Bar(name='2023현재', x=DF_prefer2[DF_prefer2['구분']=='2023재학생']['현재희망진로'], y=DF_prefer2[DF_prefer2['구분']=='2023재학생']['퍼센트(%)'], marker=dict(color=px.colors.sequential.dense[5]), hovertemplate='응답 건수: %{customdata}건<extra></extra>', customdata=DF_prefer2[DF_prefer2['구분']=='2023재학생']['카운트'], text=DF_prefer2[DF_prefer2['구분']=='2023재학생']['퍼센트(%)'])
    ])

fig.update_layout(barmode='group', bargap=0.15, height=450, width=3000,
                  yaxis={'visible': False, 'showticklabels': False},
                  xaxis={'visible': True, 'showticklabels': True},
                  showlegend=True,margin=dict(l=2, r=2),
                  plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0,0,0,0)',
                  legend=dict(orientation="h", yanchor= "top", y=1.1, xanchor= "left", x=0.01))

fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside',textfont_size=7)

fig.show()

fig.write_html('희망진로비교.html')

## **문헌정보학 흥미 교육과정** (학부 재학생)

* **2021**

In [ ]:
# 괄호 내용 지우기
Ustudent_2021['문헌정보학흥미주제'] = Ustudent_2021['문헌정보학흥미주제'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

In [ ]:
DF_2021_interest = count_to(Ustudent_2021['문헌정보학흥미주제'])
DF_2021_interest = add_classes(DF_2021_interest, '2021학부재학생', '문헌정보학흥미주제', '문헌정보학흥미주제')
DF_2021_interest

,구분,문헌정보학흥미주제,카운트,퍼센트(%)
0,2021학부재학생,정보학,32,42.67
1,2021학부재학생,자료조직,14,18.67
2,2021학부재학생,도서관경영,12,16.00
3,2021학부재학생,서지학,8,10.67
4,2021학부재학생,정보서비스,7,9.33
5,2021학부재학생,기록관리학,2,2.67


* **2022**

In [ ]:
Ustudent_2022 = Ustudent_2022.dropna(subset=['문헌정보학흥미주제'])

In [ ]:
# 괄호 내용 지우기
Ustudent_2022['문헌정보학흥미주제'] = Ustudent_2022['문헌정보학흥미주제'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

In [ ]:
DF_2022_interest = count_to(Ustudent_2022['문헌정보학흥미주제'])
DF_2022_interest = add_classes(DF_2022_interest, '2022학부재학생', '문헌정보학흥미주제', '문헌정보학흥미주제')
DF_2022_interest

,구분,문헌정보학흥미주제,카운트,퍼센트(%)
0,2022학부재학생,정보학,26,38.24
1,2022학부재학생,자료조직,16,23.53
2,2022학부재학생,도서관경영,10,14.71
3,2022학부재학생,정보서비스,8,11.76
4,2022학부재학생,서지학,6,8.82
5,2022학부재학생,기록관리학,2,2.94


* **2023**

In [ ]:
# 괄호 내용 지우기
Ustudent_2023['U문헌정보학흥미주제'] = Ustudent_2023['U문헌정보학흥미주제'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

In [ ]:
DF_2023_interest = count_to(Ustudent_2023['U문헌정보학흥미주제'])
DF_2023_interest = add_classes(DF_2023_interest, '2023학부재학생', 'U문헌정보학흥미주제', '문헌정보학흥미주제')
DF_2023_interest

,구분,문헌정보학흥미주제,카운트,퍼센트(%)
0,2023학부재학생,정보학,30,44.78
1,2023학부재학생,도서관경영,14,20.90
2,2023학부재학생,자료조직,11,16.42
3,2023학부재학생,정보서비스,8,11.94
4,2023학부재학생,서지학,2,2.99
5,2023학부재학생,기록관리학,2,2.99


* **통합**

In [ ]:
DF_interest = pd.concat([DF_2021_interest, DF_2022_interest, DF_2023_interest],axis=0)
DF_interest = DF_interest.sort_values(by=['구분', '문헌정보학흥미주제'])

In [ ]:
# 문헌정보학 흥미 주제 비교 그래프
fig = go.Figure(data=[

    go.Bar(name='2021', x=DF_2021_interest['문헌정보학흥미주제'], y=DF_2021_interest['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[0]),hovertemplate = '응답 건수: %{customdata}건<extra></extra>',customdata=DF_2021_interest['카운트'], text=DF_2021_interest['퍼센트(%)']),
    go.Bar(name='2022', x=DF_2022_interest['문헌정보학흥미주제'], y=DF_2022_interest['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[2]),hovertemplate = '응답 건수: %{customdata}건<extra></extra>',customdata=DF_2022_interest['카운트'], text=DF_2022_interest['퍼센트(%)']),
    go.Bar(name='2023', x=DF_2023_interest['문헌정보학흥미주제'], y=DF_2023_interest['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[5]),hovertemplate = '응답 건수: %{customdata}건<extra></extra>',customdata=DF_2023_interest['카운트'], text=DF_2023_interest['퍼센트(%)'])

    ])

fig.update_layout(barmode='group', bargap=0.15, height=450, width=1000,
                  yaxis={'visible': False, 'showticklabels': False},
                  xaxis={'visible': True, 'showticklabels': True},
                  showlegend=True,margin=dict(l=2, r=2),
                  plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0,0,0,0)',
                  legend=dict(orientation="h", yanchor= "top", y=1.1, xanchor= "left", x=0.01))

fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside',textfont_size=8)

fig.show()

fig.write_html('문헌정보학흥미주제비교.html')

## **희망/실제 근무 분야 비교** (재학생/취준생/취업자)

In [ ]:
def add_classes_job(df, col, ex_column, new_column):
    # 데이터프레임의 인덱스 재설정
    df.reset_index(drop=True, inplace=True)
    # '구분' 열에 'col' 값을 할당
    df['구분'] = col
    per = []
    # 각 행에 대해 '카운트'의 비율을 계산하여 '퍼센트(%)' 열에 추가
    for i in range(0, len(df)):
        per.append(round(df['카운트'].loc[i] / df['카운트'].sum() * 100, 2))
    df['퍼센트(%)'] = per
    # 열 이름 변경
    df.rename(columns={ex_column: new_column}, inplace=True)
    # '퍼센트(%)' 열을 기준으로 내림차순 정렬 후 인덱스 초기화
    df.sort_values(by='퍼센트(%)', ascending=False, inplace=True)
    df.reset_index(drop = True, inplace=True)
    # 데이터 프레임 컬럼 순서 변경
    df = df[['연도', '구분', new_column, '카운트', '퍼센트(%)']]
    return df

In [ ]:
def create_chart_job(dfs, theme, name):

    # 그래프 생성
    fig = px.bar(dfs, x="퍼센트(%)", y="연도", color=theme, orientation='h', text='퍼센트(%)',
                 color_discrete_sequence=[px.colors.sequential.dense[0], px.colors.sequential.dense[1], px.colors.sequential.dense[2],
                                          px.colors.sequential.dense[3], px.colors.sequential.dense[4], px.colors.sequential.dense[5], px.colors.sequential.dense[6],
                                          px.colors.sequential.dense[7], px.colors.sequential.dense[8], px.colors.sequential.dense[9], px.colors.sequential.dense[10]],
                 custom_data=['구분', '카운트', theme], category_orders={"연도": ['2021', '2022', '2023']})

    # 레이아웃 설정
    fig.update_layout(xaxis_tickfont_size=10,
                      legend=dict(yanchor="top", y=1.25, xanchor="left", x=0.0, orientation="h", font_size=12,
                                  font_color='black'),
                      yaxis={'visible': True, 'showticklabels': True, 'title': name},
                      xaxis={'visible': False, 'showticklabels': False}, height=300,
                      showlegend=True, font=dict(size=10, color="black"), margin=dict(l=2, r=2),
                      plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor='rgba(0,0,0,0)')

    # 텍스트 설정
    fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside', textfont_size=12)
    fig.update_traces(hovertemplate='<br>%{customdata[0]} <br>%{customdata[2]} <br>응답 건수: %{customdata[1]} 건<extra></extra>')

    # 그래프 출력
    fig.show()

    # html 변환
    fig.write_html("%s.html" % name+'_연도별_(희망)근무분야비교')

In [ ]:
def create_chart_job2(df1, df2, df3, theme, name):

    # 그래프 생성
    fig = go.Figure(data=[
    go.Bar(name='2021', x=df1[theme], y=df1['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[3]),hovertemplate = '<b>2021</b> <br>응답 건수: %{customdata[0]} 건 <br>퍼센트(%): %{customdata[1]}',customdata=df1[['카운트','퍼센트(%)']],text=df1['퍼센트(%)']),
    go.Bar(name='2022', x=df2[theme], y=df2['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[5]),hovertemplate = '<b>2022</b> <br>응답 건수: %{customdata[0]} 건 <br>퍼센트(%): %{customdata[1]}',customdata=df2[['카운트','퍼센트(%)']],text=df2['퍼센트(%)']),
    go.Bar(name='2023', x=df3[theme], y=df3['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[7]),hovertemplate = '<b>2023</b> <br>응답 건수: %{customdata[0]} 건 <br>퍼센트(%): %{customdata[1]}',customdata=df3[['카운트','퍼센트(%)']],text=df3['퍼센트(%)'])
    ])

    # 레이아웃 설정
    fig.update_layout(barmode='group',
                  height=450,width=1450,
                  yaxis={'visible': False, 'showticklabels': False},
                  xaxis={'visible': True, 'showticklabels': True, 'title': name},
                  showlegend=True,
                  margin=dict(l=2, r=2),
                  plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0,0,0,0)',
                  legend=dict(
                      orientation="h",
                      yanchor= "top",
                      y=1.1,
                      xanchor= "left",
                      x=0.01
                      )
                  )

    # 텍스트 설정
    fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside', textfont_size=12)

    # 그래프 출력
    fig.show()

    # html 변환
    fig.write_html("%s.html" % name+'_연도별_(희망)근무분야비교2')

* **재학생**

In [ ]:
# 2021 재학생
DF_2021_std_job = count_to(student_2021['희망근무분야'])
DF_2021_std_job['연도'] = '2021'
# '성우','자영업' 응답을 '기타' 처리함
DF_2021_std_job = others(DF_2021_std_job, '희망근무분야', ['성우','자영업'])
DF_2021_std_job = add_classes_job(DF_2021_std_job,'재학생','희망근무분야','(희망)근무분야')
DF_2021_std_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2021,재학생,사서직,37,42.53
1,2021,재학생,사기업,23,26.44
2,2021,재학생,공기업,13,14.94
3,2021,재학생,정부(공무원),12,13.79
4,2021,재학생,기타,2,2.30


In [ ]:
# 2022 재학생
DF_2022_std_job = count_to(student_2022['희망근무분야'])
DF_2022_std_job['연도'] = '2022'
# '전문직','공기업 사기업 고민중이다','성우','자영업','연구직','프리랜서','연구','사서직 또는 사기업' 응답을 '기타' 처리함
# 해당 문항은 단일 응답 문항으로 선택지와 다른 경우에는 모두 '기타' 처리함
DF_2022_std_job = others(DF_2022_std_job, '희망근무분야', ['전문직','공기업 사기업 고민중이다','성우','자영업','연구직',
                                             '프리랜서','연구','사서직 또는 사기업'])
DF_2022_std_job = add_classes_job(DF_2022_std_job,'재학생','희망근무분야','(희망)근무분야')
DF_2022_std_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2022,재학생,사서직,32,44.44
1,2022,재학생,사기업,17,23.61
2,2022,재학생,기타,10,13.89
3,2022,재학생,정부(공무원),9,12.50
4,2022,재학생,공기업,4,5.56


In [ ]:
# 2023 재학생
DF_2023_std_job = count_to(student_2023['U희망근무분야'])
DF_2023_std_job['연도'] = '2023'
DF_2023_std_job = add_classes_job(DF_2023_std_job,'재학생','U희망근무분야','(희망)근무분야')
DF_2023_std_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2023,재학생,사기업,38,52.05
1,2023,재학생,사서직,21,28.77
2,2023,재학생,기타,7,9.59
3,2023,재학생,정부(공무원),4,5.48
4,2023,재학생,공기업,3,4.11


In [ ]:
# 재학생 통합
DF_total_std_job = pd.concat([DF_2021_std_job,DF_2022_std_job,DF_2023_std_job],ignore_index=True)
DF_total_std_job = sorted_df_year(DF_total_std_job,'(희망)근무분야')
DF_total_std_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2021,재학생,공기업,13,14.94
1,2021,재학생,사기업,23,26.44
2,2021,재학생,사서직,37,42.53
3,2021,재학생,정부(공무원),12,13.79
4,2021,재학생,기타,2,2.30
5,2022,재학생,공기업,4,5.56
6,2022,재학생,사기업,17,23.61
7,2022,재학생,사서직,32,44.44
8,2022,재학생,정부(공무원),9,12.50
9,2022,재학생,기타,10,13.89


In [ ]:
create_chart_job(DF_total_std_job,'(희망)근무분야','재학생')

* **취준생**

In [ ]:
# 2021 취준생
DF_2021_seek_job = count_to(seeker_2021['희망근무분야3'])
DF_2021_seek_job['연도'] = '2021'
# '방송국','미정' 응답을 '기타' 처리함
DF_2021_seek_job = others(DF_2021_seek_job, '희망근무분야3', ['방송국','미정'])
DF_2021_seek_job = add_classes_job(DF_2021_seek_job,'취준생','희망근무분야3','(희망)근무분야')
DF_2021_seek_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2021,취준생,사서직,3,33.33
1,2021,취준생,사기업,3,33.33
2,2021,취준생,기타,2,22.22
3,2021,취준생,공기업,1,11.11


In [ ]:
# 2022 취준생
DF_2022_seek_job = count_to(seeker_2022['희망근무분야3'])
DF_2022_seek_job['연도'] = '2022'
# '연구원/교수' 응답을 '기타' 처리함
DF_2022_seek_job = others(DF_2022_seek_job, '희망근무분야3', ['연구원/교수'])
DF_2022_seek_job = add_classes_job(DF_2022_seek_job,'취준생','희망근무분야3','(희망)근무분야')
DF_2022_seek_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2022,취준생,공기업,2,40.0
1,2022,취준생,사기업,2,40.0
2,2022,취준생,기타,1,20.0


In [ ]:
# 2023 취준생
# 추가 조사 과정에서 응답받지 못한 경우 제외
seeker_2023_job=seeker_2023.copy()
seeker_2023_job.drop(seeker_2023_job[seeker_2023_job['index'] == 45].index, inplace=True)
seeker_2023_job

DF_2023_seek_job = count_to(seeker_2023_job['S희망근무분야'])
DF_2023_seek_job['연도'] = '2023'
DF_2023_seek_job = add_classes_job(DF_2023_seek_job,'취준생','S희망근무분야','(희망)근무분야')
DF_2023_seek_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2023,취준생,기타,3,50.0
1,2023,취준생,사서직,3,50.0


In [ ]:
# 취준생 통합
DF_total_seek_job = pd.concat([DF_2021_seek_job,DF_2022_seek_job,DF_2023_seek_job],ignore_index=True)
DF_total_seek_job = sorted_df_year(DF_total_seek_job,'(희망)근무분야')
DF_total_seek_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2021,취준생,공기업,1,11.11
1,2021,취준생,사기업,3,33.33
2,2021,취준생,사서직,3,33.33
3,2021,취준생,기타,2,22.22
4,2022,취준생,공기업,2,40.00
5,2022,취준생,사기업,2,40.00
6,2022,취준생,기타,1,20.00
7,2023,취준생,사서직,3,50.00
8,2023,취준생,기타,3,50.00


In [ ]:
create_chart_job(DF_total_seek_job,'(희망)근무분야','취준생')

* **취업자**

In [ ]:
# 2021 취업자
DF_2021_work_job = count_to(worker_2021['현재근무분야'])
DF_2021_work_job['연도'] = '2021'
# '사서교사' 응답을 '사서직' 처리함
DF_2021_work_job.drop(index=DF_2021_work_job[DF_2021_work_job['현재근무분야']=='사서교사'].index,inplace=True)
DF_2021_work_job.loc[DF_2021_work_job[DF_2021_work_job['현재근무분야']=='사서직'].index,'카운트']+=1
DF_2021_work_job = add_classes_job(DF_2021_work_job,'취업자','현재근무분야','(희망)근무분야')
DF_2021_work_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2021,취업자,사서직,8,50.00
1,2021,취업자,사기업,7,43.75
2,2021,취업자,공기업,1,6.25


In [ ]:
# 2022 취업자
DF_2022_work_job = count_to(worker_2022['현재근무분야'])
DF_2022_work_job['연도'] = '2022'
DF_2022_work_job = add_classes_job(DF_2022_work_job,'취업자','현재근무분야','(희망)근무분야')
DF_2022_work_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2022,취업자,사서직,6,66.67
1,2022,취업자,사기업,3,33.33


In [ ]:
# 2023 취업자
DF_2023_work_job = count_to(worker_2023['W근무분야'])
DF_2023_work_job['연도'] = '2023'
DF_2023_work_job = add_classes_job(DF_2023_work_job,'취업자','W근무분야','(희망)근무분야')
DF_2023_work_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2023,취업자,사서직,10,71.43
1,2023,취업자,기타,2,14.29
2,2023,취업자,사기업,2,14.29


In [ ]:
# 취업자 통합
DF_total_work_job = pd.concat([DF_2021_work_job,DF_2022_work_job,DF_2023_work_job],ignore_index=True)
DF_total_work_job = sorted_df_year(DF_total_work_job,'(희망)근무분야')
DF_total_work_job

,연도,구분,(희망)근무분야,카운트,퍼센트(%)
0,2021,취업자,공기업,1,6.25
1,2021,취업자,사기업,7,43.75
2,2021,취업자,사서직,8,50.00
3,2022,취업자,사기업,3,33.33
4,2022,취업자,사서직,6,66.67
5,2023,취업자,사기업,2,14.29
6,2023,취업자,사서직,10,71.43
7,2023,취업자,기타,2,14.29


In [ ]:
create_chart_job(DF_total_work_job,'(희망)근무분야','취업자')

## **대학원 진학 희망 여부 비교** (학부 재학생)

In [ ]:
def create_chart_grad(dfs, theme, name):

    # 그래프 생성
    fig = px.bar(dfs, x="퍼센트(%)", y="연도", color=theme, orientation='h', text='퍼센트(%)',
                 color_discrete_sequence=[px.colors.sequential.RdBu[8],px.colors.sequential.dense[2]],
                 custom_data=['구분', '카운트', theme], category_orders={"연도": ['2021', '2022', '2023']})

    # 레이아웃 설정
    fig.update_layout(xaxis_tickfont_size=10,
                      legend=dict(yanchor="top", y=1.25, xanchor="left", x=0.0, orientation="h", font_size=12,
                                  font_color='black'),
                      yaxis={'visible': True, 'showticklabels': True, 'title': name},
                      xaxis={'visible': False, 'showticklabels': False}, height=300,
                      showlegend=True, font=dict(size=10, color="black"), margin=dict(l=2, r=2),
                      plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor='rgba(0,0,0,0)')

    # 텍스트 설정
    fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside', textfont_size=12)
    fig.update_traces(hovertemplate='<br>%{customdata[0]} <br>%{customdata[2]} <br>응답 건수: %{customdata[1]} 건<extra></extra>')

    # 그래프 출력
    fig.show()

    # html 변환
    fig.write_html("%s.html" % name+'_연도별_대학원진학희망여부비교')

In [ ]:
def create_chart_grad2(df1, df2, df3, theme, name):

    # 그래프 생성
    fig = go.Figure(data=[
    go.Bar(name='2021', x=df1[theme], y=df1['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[3]),hovertemplate = '<b>2021</b> <br>응답 건수: %{customdata[0]} 건 <br>퍼센트(%): %{customdata[1]}',customdata=df1[['카운트','퍼센트(%)']],text=df1['퍼센트(%)']),
    go.Bar(name='2022', x=df2[theme], y=df2['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[5]),hovertemplate = '<b>2022</b> <br>응답 건수: %{customdata[0]} 건 <br>퍼센트(%): %{customdata[1]}',customdata=df2[['카운트','퍼센트(%)']],text=df2['퍼센트(%)']),
    go.Bar(name='2023', x=df3[theme], y=df3['퍼센트(%)'],marker=dict(color=px.colors.sequential.dense[7]),hovertemplate = '<b>2023</b> <br>응답 건수: %{customdata[0]} 건 <br>퍼센트(%): %{customdata[1]}',customdata=df3[['카운트','퍼센트(%)']],text=df3['퍼센트(%)'])
    ])

    # 레이아웃 설정
    fig.update_layout(barmode='group',
                  height=500,width=1000,
                  yaxis={'visible': False, 'showticklabels': False},
                  xaxis={'visible': True, 'showticklabels': True, 'title': name},
                  showlegend=True,
                  margin=dict(l=2, r=2),
                  plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0,0,0,0)',
                  legend=dict(
                      orientation="h",
                      yanchor= "top",
                      y=1.1,
                      xanchor= "left",
                      x=0.01
                      )
                  )

    # 텍스트 설정
    fig.update_traces(texttemplate='%{text:.2f}%', textposition='inside', textfont_size=12)

    # 그래프 출력
    fig.show()

    # html 변환
    fig.write_html("%s.html" % name+'_연도별_대학원진학희망여부비교2')

* **2021**

In [ ]:
DF_2021_grad = count_to(Ustudent_2021['대학원진학희망_여부'])
DF_2021_grad ['연도'] = '2021'
# '예 (현재 대학원 재학 중일 경우도 포함)' 응답을 '예'로 변경
DF_2021_grad['대학원진학희망_여부'] = DF_2021_grad['대학원진학희망_여부'].replace('예 (현재 대학원 재학 중일 경우도 포함)', '예')
DF_2021_grad  = add_classes_job(DF_2021_grad ,'학부재학생','대학원진학희망_여부','대학원진학희망여부')
DF_2021_grad

,연도,구분,대학원진학희망여부,카운트,퍼센트(%)
0,2021,학부재학생,아니오,52,69.33
1,2021,학부재학생,예,23,30.67


* **2022**

In [ ]:
# 2022 학부 재학생
DF_2022_grad = count_to(Ustudent_2022['대학원진학희망_여부'])
DF_2022_grad ['연도'] = '2022'
# '예 (현재 대학원 재학 중일 경우도 포함)' 응답을 '예'로 변경
DF_2022_grad['대학원진학희망_여부'] = DF_2022_grad['대학원진학희망_여부'].replace('예 (현재 대학원 재학 중일 경우도 포함)', '예')
DF_2022_grad  = add_classes_job(DF_2022_grad ,'학부재학생','대학원진학희망_여부','대학원진학희망여부')
DF_2022_grad

,연도,구분,대학원진학희망여부,카운트,퍼센트(%)
0,2022,학부재학생,아니오,50,73.53
1,2022,학부재학생,예,18,26.47


* **2023**

In [ ]:
# 2023 학부 재학생
DF_2023_grad = count_to(Ustudent_2023['U대학원진학희망여부'])
DF_2023_grad ['연도'] = '2023'
DF_2023_grad  = add_classes_job(DF_2023_grad ,'학부재학생','U대학원진학희망여부','대학원진학희망여부')
DF_2023_grad

,연도,구분,대학원진학희망여부,카운트,퍼센트(%)
0,2023,학부재학생,아니오,50,74.63
1,2023,학부재학생,예,17,25.37


* **통합**

In [ ]:
# 학부 재학생 통합
DF_total_grad = pd.concat([DF_2021_grad,DF_2022_grad,DF_2023_grad],ignore_index=True)
DF_total_grad = DF_total_grad.sort_values(by=['연도', '대학원진학희망여부'], ascending=[True, False])
DF_total_grad

,연도,구분,대학원진학희망여부,카운트,퍼센트(%)
1,2021,학부재학생,예,23,30.67
0,2021,학부재학생,아니오,52,69.33
3,2022,학부재학생,예,18,26.47
2,2022,학부재학생,아니오,50,73.53
5,2023,학부재학생,예,17,25.37
4,2023,학부재학생,아니오,50,74.63


In [ ]:
create_chart_grad(DF_total_grad, '대학원진학희망여부', '학부재학생')